In [1]:
# CONFIGURE THE MODIFIED LRG (MLRG) NETWORK (N.B.: This cell must be run before any of the training and inference cells below)

import numpy
import h5py
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import tensorflow.keras as keras
# from metric_loss_ops import triplet_semihard_loss

def loadFromH5(filename, load_labels=True):
	f = h5py.File(filename,'r')
	all_points = f['points'][:]
	count_room = f['count_room'][:]
	tmp_points = []
	idp = 0
	for i in range(len(count_room)):
		tmp_points.append(all_points[idp:idp+count_room[i], :])
		idp += count_room[i]
	f.close()
	room = []
	labels = []
	class_labels = []
	if load_labels:
		for i in range(len(tmp_points)):
			room.append(tmp_points[i][:,:-2])
			labels.append(tmp_points[i][:,-2].astype(int))
			class_labels.append(tmp_points[i][:,-1].astype(int))
		return room, labels, class_labels
	else:
		return tmp_points
    
def loadFromH5Mod(filename):
	f = h5py.File(filename,'r')
	all_points = f['points'][:]
	count_room = f['count_room'][:]
	tmp_points = []
	idp = 0
	for i in range(len(count_room)):
		tmp_points.append(all_points[idp:idp+count_room[i], :])
		idp += count_room[i]
	f.close()
	room = []
	labels = []
	class_labels = []
	frame_labels = []
	for i in range(len(tmp_points)):
		room.append(tmp_points[i][:,:-3])
		labels.append(tmp_points[i][:,-3].astype(int))
		class_labels.append(tmp_points[i][:,-2].astype(int))
		frame_labels.append(tmp_points[i][:,-1].astype(int))
	return room, labels, class_labels, frame_labels

def savePCD(filename,points):
	if len(points)==0:
		return
	f = open(filename,"w")
	l = len(points)
	header = """# .PCD v0.7 - Point Cloud Data file format
VERSION 0.7
FIELDS x y z rgb
SIZE 4 4 4 4
TYPE F F F I
COUNT 1 1 1 1
WIDTH %d
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS %d
DATA ascii
""" % (l,l)
	f.write(header)
	for p in points:
		rgb = (int(p[3]) << 16) | (int(p[4]) << 8) | int(p[5])
		f.write("%f %f %f %d\n"%(p[0],p[1],p[2],rgb))
	f.close()
	print('Saved %d points to %s' % (l,filename))

class LrgNet:
	def __init__(self,batch_size, seq_len, num_inlier_points, num_neighbor_points, feature_size, lite=1):
		if lite==0 or lite is None:
			CONV_CHANNELS = [64,64,64,128,512]
			CONV2_CHANNELS = [256, 128]
		elif lite==1:
			CONV_CHANNELS = [64,64]
			CONV2_CHANNELS = [64]
		elif lite==2:
			CONV_CHANNELS = [64,64,256]
			CONV2_CHANNELS = [64,64]
		self.kernel = [None]*len(CONV_CHANNELS)
		self.bias = [None]*len(CONV_CHANNELS)
		self.conv = [None]*len(CONV_CHANNELS)
		self.neighbor_kernel = [None]*len(CONV_CHANNELS)
		self.neighbor_bias = [None]*len(CONV_CHANNELS)
		self.neighbor_conv = [None]*len(CONV_CHANNELS)
		self.add_kernel = [None]*(len(CONV2_CHANNELS) + 1)
		self.add_bias = [None]*(len(CONV2_CHANNELS) + 1)
		self.add_conv = [None]*(len(CONV2_CHANNELS) + 1)
		self.remove_kernel = [None]*(len(CONV2_CHANNELS) + 1)
		self.remove_bias = [None]*(len(CONV2_CHANNELS) + 1)
		self.remove_conv = [None]*(len(CONV2_CHANNELS) + 1)
		self.motion_kernel = [None]*(len(CONV2_CHANNELS) + 1)
		self.motion_bias = [None]*(len(CONV2_CHANNELS) + 1)
		self.motion_conv = [None]*(len(CONV2_CHANNELS) + 1)
		self.inlier_tile = [None]*2
		self.neighbor_tile = [None]*2
		self.inlier_pl = tf.compat.v1.placeholder(tf.float32, shape=(batch_size*seq_len, num_inlier_points, feature_size))
		self.neighbor_pl = tf.compat.v1.placeholder(tf.float32, shape=(batch_size*seq_len, num_neighbor_points, feature_size))
		self.add_mask_pl = tf.compat.v1.placeholder(tf.int32, shape=(batch_size*seq_len, num_neighbor_points))
		self.remove_mask_pl = tf.compat.v1.placeholder(tf.int32, shape=(batch_size*seq_len, num_inlier_points))
        
		self.motion_mask_pl = tf.compat.v1.placeholder(tf.int32, shape=(batch_size*seq_len, num_inlier_points+num_neighbor_points))

		#CONVOLUTION LAYERS FOR INLIER SET
		for i in range(len(CONV_CHANNELS)):
			self.kernel[i] = tf.compat.v1.get_variable('lrg_kernel'+str(i), [1, feature_size if i==0 else CONV_CHANNELS[i-1], CONV_CHANNELS[i]], initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"), dtype=tf.float32)
			self.bias[i] = tf.compat.v1.get_variable('lrg_bias'+str(i), [CONV_CHANNELS[i]], initializer=tf.compat.v1.constant_initializer(0.0), dtype=tf.float32)
			self.conv[i] = tf.nn.conv1d(input=self.inlier_pl if i==0 else self.conv[i-1], filters=self.kernel[i], stride=1, padding='VALID')
			self.conv[i] = tf.nn.bias_add(self.conv[i], self.bias[i])
			self.conv[i] = tf.nn.relu(self.conv[i])

		#CONVOLUTION LAYERS FOR NEIGHBOR SET
		for i in range(len(CONV_CHANNELS)):
			self.neighbor_kernel[i] = tf.compat.v1.get_variable('lrg_neighbor_kernel'+str(i), [1, feature_size if i==0 else CONV_CHANNELS[i-1], CONV_CHANNELS[i]], initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"), dtype=tf.float32)
			self.neighbor_bias[i] = tf.compat.v1.get_variable('lrg_neighbor_bias'+str(i), [CONV_CHANNELS[i]], initializer=tf.compat.v1.constant_initializer(0.0), dtype=tf.float32)
			self.neighbor_conv[i] = tf.nn.conv1d(input=self.neighbor_pl if i==0 else self.neighbor_conv[i-1], filters=self.neighbor_kernel[i], stride=1, padding='VALID')
			self.neighbor_conv[i] = tf.nn.bias_add(self.neighbor_conv[i], self.neighbor_bias[i])
			self.neighbor_conv[i] = tf.nn.relu(self.neighbor_conv[i])

		#MAX POOLING
		self.pool = tf.reduce_max(input_tensor=self.conv[-1], axis=1)
		self.neighbor_pool = tf.reduce_max(input_tensor=self.neighbor_conv[-1], axis=1)
		self.combined_pool = tf.concat(axis=1, values=[self.pool, self.neighbor_pool])
		self.pooled_feature = self.combined_pool

		#CONCAT AFTER POOLING
		self.inlier_tile[0] = tf.tile(tf.reshape(self.pooled_feature,[batch_size*seq_len,-1,CONV_CHANNELS[-1]*2]) , [1,1,num_inlier_points])
		self.inlier_tile[0] = tf.reshape(self.inlier_tile[0],[batch_size*seq_len,num_inlier_points,-1])
		self.inlier_tile[1] = self.conv[1]
		self.inlier_concat = tf.concat(axis=2, values=self.inlier_tile)
		self.neighbor_tile[0] = tf.tile(tf.reshape(self.pooled_feature,[batch_size*seq_len,-1,CONV_CHANNELS[-1]*2]) , [1,1,num_neighbor_points])
		self.neighbor_tile[0] = tf.reshape(self.neighbor_tile[0],[batch_size*seq_len,num_neighbor_points,-1])
		self.neighbor_tile[1] = self.neighbor_conv[1]
		self.neighbor_concat = tf.concat(axis=2, values=self.neighbor_tile)
        
		self.all_concat = tf.concat(values = [self.inlier_concat,self.neighbor_concat],axis=1)

		#CONVOLUTION LAYERS AFTER POOLING
		for i in range(len(CONV2_CHANNELS)):
			self.add_kernel[i] = tf.compat.v1.get_variable('lrg_add_kernel'+str(i), [1, CONV_CHANNELS[-1]*2 + CONV_CHANNELS[1] if i==0 else CONV2_CHANNELS[i-1], CONV2_CHANNELS[i]], initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"), dtype=tf.float32)
			self.add_bias[i] = tf.compat.v1.get_variable('lrg_add_bias'+str(i), [CONV2_CHANNELS[i]], initializer=tf.compat.v1.constant_initializer(0.0), dtype=tf.float32)
			self.add_conv[i] = tf.nn.conv1d(input=self.neighbor_concat if i==0 else self.add_conv[i-1], filters=self.add_kernel[i], stride=1, padding='VALID')
			self.add_conv[i] = tf.nn.bias_add(self.add_conv[i], self.add_bias[i])
			self.add_conv[i] = tf.nn.relu(self.add_conv[i])
		i += 1
		self.add_kernel[i] = tf.compat.v1.get_variable('lrg_add_kernel'+str(i), [1, CONV2_CHANNELS[-1], 2], initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"), dtype=tf.float32)
		self.add_bias[i] = tf.compat.v1.get_variable('lrg_add_bias'+str(i), [2], initializer=tf.compat.v1.constant_initializer(0.0), dtype=tf.float32)
		self.add_conv[i] = tf.nn.conv1d(input=self.add_conv[i-1], filters=self.add_kernel[i], stride=1, padding='VALID')
		self.add_conv[i] = tf.nn.bias_add(self.add_conv[i], self.add_bias[i])
		self.add_output = self.add_conv[i]

		for i in range(len(CONV2_CHANNELS)):
			self.remove_kernel[i] = tf.compat.v1.get_variable('lrg_remove_kernel'+str(i), [1, CONV_CHANNELS[-1]*2 + CONV_CHANNELS[1] if i==0 else CONV2_CHANNELS[i-1], CONV2_CHANNELS[i]], initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"), dtype=tf.float32)
			self.remove_bias[i] = tf.compat.v1.get_variable('lrg_remove_bias'+str(i), [CONV2_CHANNELS[i]], initializer=tf.compat.v1.constant_initializer(0.0), dtype=tf.float32)
			self.remove_conv[i] = tf.nn.conv1d(input=self.inlier_concat if i==0 else self.remove_conv[i-1], filters=self.remove_kernel[i], stride=1, padding='VALID')
			self.remove_conv[i] = tf.nn.bias_add(self.remove_conv[i], self.remove_bias[i])
			self.remove_conv[i] = tf.nn.relu(self.remove_conv[i])
		i += 1
		self.remove_kernel[i] = tf.compat.v1.get_variable('lrg_remove_kernel'+str(i), [1, CONV2_CHANNELS[-1], 2], initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"), dtype=tf.float32)
		self.remove_bias[i] = tf.compat.v1.get_variable('lrg_remove_bias'+str(i), [2], initializer=tf.compat.v1.constant_initializer(0.0), dtype=tf.float32)
		self.remove_conv[i] = tf.nn.conv1d(input=self.remove_conv[i-1], filters=self.remove_kernel[i], stride=1, padding='VALID')
		self.remove_conv[i] = tf.nn.bias_add(self.remove_conv[i], self.remove_bias[i])
		self.remove_output = self.remove_conv[i]
        
		for i in range(len(CONV2_CHANNELS)):
			self.motion_kernel[i] = tf.compat.v1.get_variable('lrg_motion_kernel'+str(i), [1, CONV_CHANNELS[-1]*2 + CONV_CHANNELS[1] if i==0 else CONV2_CHANNELS[i-1], CONV2_CHANNELS[i]], initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"), dtype=tf.float32)
			self.motion_bias[i] = tf.compat.v1.get_variable('lrg_motion_bias'+str(i), [CONV2_CHANNELS[i]], initializer=tf.compat.v1.constant_initializer(0.0), dtype=tf.float32)
			self.motion_conv[i] = tf.nn.conv1d(input=self.all_concat if i==0 else self.motion_conv[i-1], filters=self.motion_kernel[i], stride=1, padding='VALID')
			self.motion_conv[i] = tf.nn.bias_add(self.motion_conv[i], self.motion_bias[i])
			self.motion_conv[i] = tf.nn.relu(self.motion_conv[i])
		i += 1
		self.motion_kernel[i] = tf.compat.v1.get_variable('lrg_motion_kernel'+str(i), [1, CONV2_CHANNELS[-1], 2], initializer=tf.keras.initializers.VarianceScaling(scale=1.0, mode="fan_avg", distribution="uniform"), dtype=tf.float32)
		self.motion_bias[i] = tf.compat.v1.get_variable('lrg_motion_bias'+str(i), [2], initializer=tf.compat.v1.constant_initializer(0.0), dtype=tf.float32)
		self.motion_conv[i] = tf.nn.conv1d(input=self.motion_conv[i-1], filters=self.motion_kernel[i], stride=1, padding='VALID')
		self.motion_conv[i] = tf.nn.bias_add(self.motion_conv[i], self.motion_bias[i])
		self.motion_output = self.motion_conv[i]

		#LOSS FUNCTIONS
		def weighted_cross_entropy(logit, label):
			pos_mask = tf.compat.v1.where(tf.cast(label, tf.bool))
			neg_mask = tf.compat.v1.where(tf.cast(1 - label, tf.bool))
			pos_loss = tf.reduce_mean(input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=tf.gather_nd(logit, pos_mask), labels=tf.gather_nd(label, pos_mask)))
			neg_loss = tf.reduce_mean(input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=tf.gather_nd(logit, neg_mask), labels=tf.gather_nd(label, neg_mask)))
			pos_loss = tf.cond(pred=tf.math.is_nan(pos_loss), true_fn=lambda: 0.0, false_fn=lambda: pos_loss)
			neg_loss = tf.cond(pred=tf.math.is_nan(neg_loss), true_fn=lambda: 0.0, false_fn=lambda: neg_loss)
			return pos_loss + neg_loss

		self.add_loss = tf.reduce_mean(input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.add_output, labels=self.add_mask_pl))
		self.add_acc = tf.reduce_mean(input_tensor=tf.cast(tf.equal(tf.argmax(input=self.add_output, axis=-1), tf.cast(self.add_mask_pl, dtype=tf.int64)), tf.float32))
		TP = tf.reduce_sum(input_tensor=tf.cast(tf.logical_and(tf.equal(tf.argmax(input=self.add_output, axis=-1), 1), tf.equal(self.add_mask_pl, 1)), tf.float32))
		self.add_prc = TP / (tf.cast(tf.reduce_sum(input_tensor=tf.argmax(input=self.add_output, axis=-1)), tf.float32) + 1)
		self.add_rcl = TP / (tf.cast(tf.reduce_sum(input_tensor=self.add_mask_pl), tf.float32) + 1)

		self.remove_loss = weighted_cross_entropy(self.remove_output, self.remove_mask_pl)
		self.remove_acc = tf.reduce_mean(input_tensor=tf.cast(tf.equal(tf.argmax(input=self.remove_output, axis=-1), tf.cast(self.remove_mask_pl, dtype=tf.int64)), tf.float32))
		self.remove_mask = tf.nn.softmax(self.remove_output, axis=-1)[:, :, 1] > 0.5
		TP = tf.reduce_sum(input_tensor=tf.cast(tf.logical_and(self.remove_mask, tf.equal(self.remove_mask_pl, 1)), tf.float32))
		self.remove_prc = TP / (tf.reduce_sum(input_tensor=tf.cast(self.remove_mask, tf.float32)) + 1)
		self.remove_rcl = TP / (tf.cast(tf.reduce_sum(input_tensor=self.remove_mask_pl), tf.float32) + 1)
        
		self.motion_loss = weighted_cross_entropy(self.motion_output, self.motion_mask_pl)
		self.motion_acc = tf.reduce_mean(input_tensor=tf.cast(tf.equal(tf.argmax(input=self.motion_output, axis=-1), tf.cast(self.motion_mask_pl, dtype=tf.int64)), tf.float32))
		self.motion_mask = tf.nn.softmax(self.motion_output, axis=-1)[:, :, 1] > 0.5
		TP = tf.reduce_sum(input_tensor=tf.cast(tf.logical_and(self.motion_mask, tf.equal(self.motion_mask_pl, 1)), tf.float32))
		self.motion_prc = TP / (tf.reduce_sum(input_tensor=tf.cast(self.motion_mask, tf.float32)) + 1)
		self.motion_rcl = TP / (tf.cast(tf.reduce_sum(input_tensor=self.motion_mask_pl), tf.float32) + 1)

		self.loss = self.add_loss + self.remove_loss + self.motion_loss
		batch = tf.Variable(0)
		optimizer = tf.compat.v1.train.AdamOptimizer(1e-3)
		self.train_op = optimizer.minimize(self.loss, global_step=batch)

tf.compat.v1.disable_eager_execution()

# BATCH_SIZE = 2
# NUM_INLIER_POINT = 512
# NUM_NEIGHBOR_POINT = 512
# FEATURE_SIZE = 11
# LITE = None
# config = tf.compat.v1.ConfigProto()
# config.gpu_options.allow_growth = True
# config.allow_soft_placement = True
# config.log_device_placement = False
# sess = tf.compat.v1.Session(config=config)
# net = LrgNet(BATCH_SIZE, 1, NUM_INLIER_POINT, NUM_NEIGHBOR_POINT, FEATURE_SIZE, LITE)

# init = tf.compat.v1.global_variables_initializer()
# sess.run(init, {})

# @tf.function
# def traceme():#inlier_points,neighbor_points,input_add,input_remove):
#     inlier_points = numpy.zeros((BATCH_SIZE, NUM_INLIER_POINT, FEATURE_SIZE))
#     neighbor_points = numpy.zeros((BATCH_SIZE, NUM_NEIGHBOR_POINT, FEATURE_SIZE))
#     input_add = numpy.zeros((BATCH_SIZE, NUM_NEIGHBOR_POINT), dtype=numpy.int32)
#     input_remove = numpy.zeros((BATCH_SIZE, NUM_INLIER_POINT), dtype=numpy.int32)
    
#     ls, ap, ar, rp, rr = sess.run([net.loss, net.add_prc, net.add_rcl, net.remove_prc, net.remove_rcl],
#     {net.inlier_pl:inlier_points, net.neighbor_pl:neighbor_points, net.add_mask_pl:input_add, net.remove_mask_pl:input_remove})
#     return ls, ap, ar, rp, rr



# # ls, ap, ar, rp, rr = traceme(inlier_points_test,neighbor_points_test,input_add_test,input_remove_test)

# logdir = "logs"
# writer = tf.summary.create_file_writer(logdir)
# tf.summary.trace_on(graph=True, profiler=True)
# # Forward pass
# traceme()#inlier_points_test,neighbor_points_test,input_add_test,input_remove_test)
# with writer.as_default():
#     tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=logdir)

In [ ]:
# STAGE SEMANTIC KITTI MODIFIED

import numpy as np
import h5py
import sys
import os
import itertools
import networkx as nx
# import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
# import matplotlib.animation as animation

INTERVAL = 20
SKIP = 0
VOXEL_RESOLUTION = 0.3
DOWNSAMPLE_RESOLUTION = 0.3 #0.1
MIN_CLUSTER = 50

sequence='07'

class Track:
    def __init__(self, obj_id):
        self.obj_id = obj_id
        self.centroids = []
        self.frames = []
        self.pointcount = []
        self.length = 0
        self.moving = False
        self.motion = []
        self.averagemotion = 0
        
    def Extend(self,centroid,frame,pointcount):
        self.centroids.append(centroid)
        self.frames.append(frame)
        self.pointcount.append(pointcount)
        self.length += 1

def downsample(cloud, resolution):
    voxel_coordinates = [tuple(p) for p in np.round((cloud[:,:3] / resolution)).astype(int)]
    voxel_set = set()
    downsampled_cloud = []
    for i in range(len(cloud)):
        if not voxel_coordinates[i] in voxel_set:
            voxel_set.add(voxel_coordinates[i])
            downsampled_cloud.append(cloud[i])
    return np.array(downsampled_cloud)

def sample_sphere(radius,trans,npts):
    sphere = []
    azis=(np.random.random(npts)-0.5)*2*np.pi
    eles=(np.random.random(npts)-0.5)*np.pi
    for i in range(npts):
        pt=[radius*np.sin(eles[i])*np.cos(azis[i]),radius*np.sin(eles[i])*np.sin(azis[i]),radius*np.cos(eles[i])]
        for j in range(3):
            pt[j]+=trans[j]
        sphere.append(pt)
    return sphere

def sample_cube(side,trans,npts):
    cube = []
    us=np.random.random(npts)-0.5
    vs=np.random.random(npts)-0.5
    faces = np.random.randint(0,6,npts)
    
    rot1=np.random.random()*360
    rot2=np.random.random()*360
    rot3=np.random.random()*360
    r = R.from_euler('zyx', [rot1, rot2, rot3], degrees=True)
    
    for i in range(npts):
        if faces[i]==0:
            pt=[ 0.5*side,us[i]*side,vs[i]*side]
        if faces[i]==1:
            pt=[-0.5*side,us[i]*side,vs[i]*side]
        if faces[i]==2:
            pt=[us[i]*side, 0.5*side,vs[i]*side]
        if faces[i]==3:
            pt=[us[i]*side,-0.5*side,vs[i]*side]
        if faces[i]==4:
            pt=[us[i]*side,vs[i]*side, 0.5*side]
        if faces[i]==5:
            pt=[us[i]*side,vs[i]*side,-0.5*side]
        ptr=r.apply(pt)
        for j in range(3):
            pt[j]+=trans[j]
        cube.append(pt)
    return cube
    

# get camera calibration
calib_file = open('Z:\data_odometry_velodyne\dataset\sequences\\' + sequence + '\calib.txt', 'r')
calib = {}
for line in calib_file:
    key, content = line.strip().split(":")
    values = [float(v) for v in content.strip().split()]
    pose = np.zeros((4, 4))
    pose[0, 0:4] = values[0:4]
    pose[1, 0:4] = values[4:8]
    pose[2, 0:4] = values[8:12]
    pose[3, 3] = 1.0
    calib[key] = pose
calib_file.close()

# get poses
poses = []
Tr = calib["Tr"]
Tr_inv = np.linalg.inv(Tr)
pose_file = open('Z:\data_odometry_velodyne\dataset\sequences\\' + sequence + '\poses.txt', 'r')
for line in pose_file:
    values = [float(v) for v in line.strip().split()]
    pose = np.zeros((4, 4))
    pose[0, 0:4] = values[0:4]
    pose[1, 0:4] = values[4:8]
    pose[2, 0:4] = values[8:12]
    pose[3, 3] = 1.0
    poses.append(np.matmul(Tr_inv, np.matmul(pose, Tr)))
pose_file.close()

scan_paths = 'Z:\data_odometry_velodyne\dataset\sequences\\' + sequence + '\\velodyne'
scan_names = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(scan_paths)) for f in fn]
# scan_names=os.listdir(scan_paths)
scan_names.sort()
print('Scans: ',len(scan_names))
    
label_paths = 'Z:\data_odometry_velodyne\dataset\sequences\\' + sequence + '\labels'
label_names = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(label_paths)) for f in fn]
label_names.sort()
print('Labels: ',len(label_names))

stacked_points = []
counts = []

frames = len(scan_names)
offset = INTERVAL-1
tracks = []

all_points = []

obj_ids_uq = []
obj_ids_tr = []

for k in range(len(scan_names)):
    if k%50 == 0:
        print(k,' of ',len(scan_names))
    scan = np.fromfile(scan_names[k], dtype=np.float32)
    scan = scan.reshape((-1, 4))
    
    # get XYZ in world coordinate frame
    xyz_local = scan[:, 0:3]
    R_world_local = poses[k][:3, :3]
    t_world_local = poses[k][:3, 3]
    xyz_world = xyz_local.dot(R_world_local.T) + t_world_local
#     xyz_voxels = [tuple(v) for v in np.round(xyz_world / VOXEL_RESOLUTION).astype(int)]
    
    # get point labels
    label = np.fromfile(label_names[k], dtype=np.uint32)
    obj_id = [l >> 16 for l in label]
    cls_id = [l & 0xFFFF for l in label]
    
    # stack in Nx5 array
    points = np.zeros((len(xyz_world), 6))
    points[:, :3] = xyz_world
    points[:, 3] = obj_id
    points[:, 4] = cls_id
    
    points_sorted=points[points[:, 3].argsort()]
    obj_id_sorted=points_sorted[:, 3]
    unique_obj_ids=np.unique(obj_id_sorted)
    point_count=0
    
    for obj_id_unique in unique_obj_ids:
        track_index=-1
        for j in range(len(tracks)):
            if(tracks[j].obj_id==obj_id_unique):
                track_index = j
                break
        if track_index == -1:
            track_index = len(tracks)
            tracks.append(Track(obj_id_unique))
            if not(obj_id_unique in obj_ids_uq):
                obj_ids_uq.append(obj_id_unique)
                obj_ids_tr.append(track_index)

        obj_pts=points_sorted[(obj_id_sorted==obj_id_unique).astype(bool),:]
        point_count+=obj_pts.shape[0]
        obj_cent=np.mean(obj_pts, axis=0)
        tracks[track_index].Extend(obj_cent[0:3],k,obj_pts.shape[0])
        
N_TRACKS=len(tracks)

for i in range(N_TRACKS):
    if tracks[i].length >1:
        for j in range(1,tracks[i].length):
            delta = tracks[i].centroids[j]-tracks[i].centroids[j-1]
            tracks[i].motion.append(delta)
            tracks[i].averagemotion+=np.linalg.norm(delta)
        tracks[i].averagemotion/=float(tracks[i].length)
        if tracks[i].averagemotion > 2.5:
            tracks[i].moving = True
        
print(N_TRACKS, ' Unique Tracks found')
for i in range(N_TRACKS):
    if tracks[i].length > 1:
        print('Track: ',i,' with length: ',tracks[i].length,' average speed: ',tracks[i].averagemotion,
              ' average points: ',sum(tracks[i].pointcount)/float(tracks[i].length))
        
for u in range(len(obj_ids_uq)):
    print('Object ID: ',obj_ids_uq[u],' is associated with Track: ',obj_ids_tr[u])
        
colors=np.random.rand(10000,3)*255

# fig1,ax1 = plt.subplots(figsize=(10, 10))
# rot1=45
# rot2=45
# rot3=0
# r = R.from_euler('zyx', [rot1, rot2, rot3], degrees=True)
# scale = 0.125



# rgb=np.zeros((1000,1000,3),dtype=np.uint8)
# for k in range(N_TRACKS):
#     for j in range(tracks[k].length):
#         vr=r.apply([tracks[k].centroids[j][0],tracks[k].centroids[j][1],tracks[k].centroids[j][2]])
    
#         x=int(vr[0]*scale)+500
#         y=int(vr[1]*scale)+500
#         if (x>=1) and (x<999) and (y>=1) and (y<999):
#             rgb[x,y,0]=int(colors[k,0])
#             rgb[x,y,1]=int(colors[k,1])
#             rgb[x,y,2]=int(colors[k,2])
            
#             rgb[x+1,y,0]=int(colors[k,0])
#             rgb[x+1,y,1]=int(colors[k,1])
#             rgb[x+1,y,2]=int(colors[k,2])
            
#             rgb[x-1,y,0]=int(colors[k,0])
#             rgb[x-1,y,1]=int(colors[k,1])
#             rgb[x-1,y,2]=int(colors[k,2])
            
#             rgb[x,y+1,0]=int(colors[k,0])
#             rgb[x,y+1,1]=int(colors[k,1])
#             rgb[x,y+1,2]=int(colors[k,2])
            
#             rgb[x,y-1,0]=int(colors[k,0])
#             rgb[x,y-1,1]=int(colors[k,1])
#             rgb[x,y-1,2]=int(colors[k,2])
            
# im = ax1.imshow(rgb)
# plt.show()



        
while offset < frames:
    print(offset)
    
    for k in range(offset-INTERVAL+1,offset+1):
        scan = np.fromfile(scan_names[k], dtype=np.float32)
        scan = scan.reshape((-1, 4))

        # get XYZ in world coordinate frame
        xyz_local = scan[:, 0:3]
        R_world_local = poses[k][:3, :3]
        t_world_local = poses[k][:3, 3]
        xyz_world = xyz_local.dot(R_world_local.T) + t_world_local
#         xyz_voxels = [tuple(v) for v in np.round(xyz_world / VOXEL_RESOLUTION).astype(int)]
    
        # get point labels
        label = np.fromfile(label_names[k], dtype=np.uint32)
        obj_id = [l >> 16 for l in label]
        cls_id = [l & 0xFFFF for l in label]
        
        trk_id = []
        for i in obj_id:
            trk_id.append(obj_ids_tr[obj_ids_uq.index(i)])
    
        npts_sphere1 = 300
        npts_sphere2 = 300
        npts_cube1 = 300
        npts_orig = len(xyz_world)
        npts_total = npts_orig + npts_sphere1 + npts_sphere2 + npts_cube1
        # stack in Nx8 array
        points = np.zeros((npts_total, 8))
        points[0:npts_orig, :3] = xyz_world
        points[0:npts_orig, 3] = obj_id
        points[0:npts_orig, 4] = cls_id
        points[0:npts_orig, 5] = k-offset
        points[0:npts_orig, 6] = trk_id
        points[0:npts_orig, 7] = 0
        for i in range(len(xyz_world)):
            if tracks[trk_id[i]].moving:
                points[i, 7] = 1
        
        # Artificial sphere 1
        sphere1_geom = sample_sphere(1,[1.5+(np.sin(float(k)*0.1)*2.0), 5.0+(np.cos(float(k)*0.1)*2.0), 1.0],npts_sphere1)
        npts_start = npts_orig
        for i in range(npts_sphere1):
            points[npts_start+i,0:3]=sphere1_geom[i][0:3] + t_world_local
            points[npts_start+i,3]=5001
            points[npts_start+i,4]=-1
            points[npts_start+i,5]=k-offset
            points[npts_start+i,6]=5001
            points[npts_start+i,7]=1
        
        # Artificial sphere 2
        sphere2_geom = sample_sphere(1.33,[1.5+(np.sin(float(-k)*0.1)*4.0), 5.0+(np.cos(float(-k)*0.1)*4.0), 1.0],npts_sphere2)
        npts_start = npts_orig + npts_sphere1
        for i in range(npts_sphere2):
            points[npts_start+i,0:3]=sphere2_geom[i][0:3] + t_world_local
            points[npts_start+i,3]=5002
            points[npts_start+i,4]=-1
            points[npts_start+i,5]=k-offset
            points[npts_start+i,6]=5002
            points[npts_start+i,7]=1

        # Artificial cube 1
        cube1_geom = sample_cube(1.33,[1.5+(np.sin(float(k)*0.1)*6.0), 5.0+(np.cos(float(k)*0.1)*6.0), 1.0],npts_cube1)
        npts_start = npts_orig + npts_sphere1 + npts_sphere2
        for i in range(npts_cube1):
            points[npts_start+i,0:3]=cube1_geom[i][0:3] + t_world_local
            points[npts_start+i,3]=5003
            points[npts_start+i,4]=-1
            points[npts_start+i,5]=k-offset
            points[npts_start+i,6]=5003
            points[npts_start+i,7]=1
        
        stacked_points.extend(points)
    
    stacked_points = np.array(stacked_points)
    stacked_points = downsample(stacked_points, DOWNSAMPLE_RESOLUTION)
    
    equalized_idx = []
    unequalized_idx = []
    equalized_map = {}
    point_voxels = [tuple(v) for v in np.round(stacked_points[:,:3]/VOXEL_RESOLUTION).astype(int)]
    for i in range(len(stacked_points)):
        k = point_voxels[i]
        if not k in equalized_map:
            equalized_map[k] = len(equalized_idx)
            equalized_idx.append(i)
        unequalized_idx.append(equalized_map[k])
    points = stacked_points[equalized_idx, :]
    point_voxels = [tuple(v) for v in np.round(points[:,:3]/VOXEL_RESOLUTION).astype(int)]
    obj_id = points[:, 3]
    cls_id = points[:, 4]
    new_obj_id = np.zeros(len(obj_id), dtype=int)

    # connected components to label unassigned obj IDs
    original_obj_id = set(points[:, 3]) - set([0])
    cluster_id = 1
    for i in original_obj_id:
        new_obj_id[obj_id == i] = cluster_id
        cluster_id += 1 

    edges = []
    for i in range(len(point_voxels)):
        if obj_id[i] > 0:
            continue
        k = point_voxels[i]
        for d in itertools.product([-1,0,1],[-1,0,1],[-1,0,1]):
            if d!=(0,0,0):
                kk = (k[0]+d[0], k[1]+d[1], k[2]+d[2])
                if kk in equalized_map and cls_id[i] == cls_id[equalized_map[kk]]:
                    edges.append([i, equalized_map[kk]])
    G = nx.Graph(edges)
    clusters = nx.connected_components(G)
    clusters = [list(c) for c in clusters]
    for i in range(len(clusters)):
        if len(clusters[i]) > MIN_CLUSTER:
            new_obj_id[clusters[i]] = cluster_id
            cluster_id += 1

    stacked_points[:, 3] = new_obj_id[unequalized_idx]
    stacked_points = stacked_points[stacked_points[:, 3] > 0, :]

    print('Processing %d points from %s'%(len(stacked_points), scan_names[offset][:]))
    all_points.extend(stacked_points)
    counts.append(len(stacked_points))
    stacked_points = []

    offset += 1

h5_fout = h5py.File('Z:\PHD\pcseg\KITTI_MOD\KITTI_I20_' + sequence + '.h5','w')
h5_fout.create_dataset('points', data=all_points, compression='gzip', compression_opts=4, dtype=np.float32)
h5_fout.create_dataset('count_room', data=counts, compression='gzip', compression_opts=4, dtype=np.int32)
h5_fout.close()

In [ ]:
# STAGE DATA MODIFIED V2

import h5py
import itertools
import sys
import numpy as np

def loadFromH5ModV2(filename):
	f = h5py.File(filename,'r')
	all_points = f['points'][:]
	count_room = f['count_room'][:]
	tmp_points = []
	idp = 0
	for i in range(len(count_room)):
		tmp_points.append(all_points[idp:idp+count_room[i], :])
		idp += count_room[i]
	f.close()
	room = []
	labels = []
	class_labels = []
	frame_labels = []
	motion_labels = []
	for i in range(len(tmp_points)):
		room.append(tmp_points[i][:,:-5])
		labels.append(tmp_points[i][:,-5].astype(int))
		class_labels.append(tmp_points[i][:,-4].astype(int))
		frame_labels.append(tmp_points[i][:,-3].astype(int))
		motion_labels.append(tmp_points[i][:,-1].astype(int))
	return room, labels, class_labels, frame_labels, motion_labels

resolution = 0.3 #0.6 #0.1
SEED = 4
cluster_threshold = 10
max_points = 1024
save_id = 0
np.random.seed(0)
AREAS = ['KITTI_I20']

for AREA in AREAS:
	all_points,all_obj_id,all_cls_id,all_frm_id,all_motion = loadFromH5ModV2('Z:\PHD\pcseg\KITTI_MOD\%s_%02d.h5' % (AREA, SEED))
	stacked_points = []
	stacked_neighbor_points = []
	stacked_count = []
	stacked_neighbor_count = []
	stacked_remove = []
	stacked_add = []
	stacked_motion = []
	stacked_steps = []
	stacked_complete = []

	for room_id in range(30): #len(all_points)):
		print('Room ID: ',room_id)

		unequalized_points = all_points[room_id]
		obj_id = all_obj_id[room_id]
		cls_id = all_cls_id[room_id]
		frm_id = all_frm_id[room_id]
		motion = all_motion[room_id]
		#normalize frame IDs

		#equalize resolution
		equalized_idx = []
		equalized_set = set()
		normal_grid = {}
		for i in range(len(unequalized_points)):
			k = tuple(np.round(unequalized_points[i,:3]/resolution).astype(int))
			if not k in equalized_set:
				equalized_set.add(k)
				equalized_idx.append(i)
			if not k in normal_grid:
				normal_grid[k] = []
			normal_grid[k].append(i)
		# points -> XYZ + RGB
		points = unequalized_points[equalized_idx]
		obj_id = obj_id[equalized_idx]
		cls_id = cls_id[equalized_idx]
		frm_id = frm_id[equalized_idx]
		motion = motion[equalized_idx]
		xyz = points[:,:3]
		rgb = points[:,3:6]
		room_coordinates = (xyz - xyz.min(axis=0)) / (xyz.max(axis=0) - xyz.min(axis=0))

		#compute normals and curvatures
		normals = []
		curvatures = []
		for i in range(len(points)):
			k = tuple(np.round(points[i,:3]/resolution).astype(int))
			neighbors = []
			for offset in itertools.product([-1,0,1],[-1,0,1],[-1,0,1]):
				kk = (k[0]+offset[0], k[1]+offset[1], k[2]+offset[2])
				if kk in normal_grid:
					neighbors.extend(normal_grid[kk])
			accA = np.zeros((3,3))
			accB = np.zeros(3)
			for n in neighbors:
				p = unequalized_points[n,:3]
				accA += np.outer(p,p)
				accB += p
			cov = accA / len(neighbors) - np.outer(accB, accB) / len(neighbors)**2
			U,S,V = np.linalg.svd(cov)
			normals.append(np.fabs(V[2]))
			curvature = S[2] / (S[0] + S[1] + S[2])
			curvatures.append(np.fabs(curvature)) 
		normals = np.array(normals)
		curvatures = np.array(curvatures)
		curvatures = curvatures/curvatures.max()

		points = np.hstack((xyz, room_coordinates, normals, curvatures.reshape(-1,1), frm_id.reshape(-1,1))).astype(np.float32)

		point_voxels = np.round(points[:,:3]/resolution).astype(int)
		visited = np.zeros(len(point_voxels), dtype=bool)

		#iterate over each voxel in the room
		for seed_id in np.random.choice(range(len(points)), len(points), replace=False):
#		for seed_id in np.arange(len(points))[np.argsort(curvatures)]:
			if visited[seed_id]:
				continue
			target_id = obj_id[seed_id]
			obj_voxels = point_voxels[obj_id==target_id, :]
# 			print(obj_voxels)
			gt_mask = obj_id==target_id
			original_minDims = obj_voxels.min(axis=0)
			original_maxDims = obj_voxels.max(axis=0)
			#print('original',np.sum(gt_mask), original_minDims, original_maxDims)
			mask = np.logical_and(np.all(point_voxels>=original_minDims,axis=1), np.all(point_voxels<=original_maxDims, axis=1))
			originalScore = 1.0 * np.sum(np.logical_and(gt_mask,mask)) / np.sum(np.logical_or(gt_mask,mask))

			#initialize the seed voxel
			seed_voxel = point_voxels[seed_id]
			currentMask = np.zeros(len(points), dtype=bool)
			currentMask[seed_id] = True
			minDims = seed_voxel.copy()
			maxDims = seed_voxel.copy()
			steps = 0
			stuck = False
			add_mistake_prob = np.random.randint(2,5)*0.1
			remove_mistake_prob = np.random.randint(2,5)*0.1
			add_mistake_limit = np.inf
			remove_mistake_limit = np.inf
#			add_mistake_prob = 0.03
#			remove_mistake_prob = 0.01
#			add_mistake_limit = 30.0
#			remove_mistake_limit = 30.0

			#perform region growing for all frames
			frm_done=False

			while not(frm_done):

				#determine the current points and the neighboring points
				currentPoints = points[currentMask, :].copy()
				newMinDims = minDims.copy()	
				newMaxDims = maxDims.copy()	
				newMinDims -= 1
				newMaxDims += 1
				mask = np.logical_and(np.all(point_voxels>=newMinDims,axis=1), np.all(point_voxels<=newMaxDims, axis=1))
				mask = np.logical_and(mask, np.logical_not(currentMask))
				mask = np.logical_and(mask, np.logical_not(visited))

				#determine which points to accept
				expandPoints = points[mask, :].copy()
				expandClass = obj_id[mask] == target_id
				expandClass_motion = motion[mask]           
				mask_idx = np.nonzero(mask)[0]
				if stuck:
					expandID = mask_idx[expandClass]
				else:
#					mistake_sample = np.random.random(len(mask_idx)) < add_mistake_prob
					mistake_sample = np.random.random(len(mask_idx)) < min(add_mistake_prob, add_mistake_limit/(len(mask_idx)+1))
					expand_with_mistake = np.logical_xor(expandClass, mistake_sample)
					expandID = mask_idx[expand_with_mistake]

				#determine which points to reject
				rejectClass = obj_id[currentMask] != target_id
				rejectClass_motion = motion[currentMask] 
				mask_idx = np.nonzero(currentMask)[0]
				if stuck:
					rejectID = mask_idx[rejectClass]
				else:
#					mistake_sample = np.random.random(len(mask_idx)) < remove_mistake_prob
					mistake_sample = np.random.random(len(mask_idx)) < min(remove_mistake_prob, remove_mistake_limit/(len(mask_idx)+1))
					reject_with_mistake = np.logical_xor(rejectClass, mistake_sample)
					rejectID = mask_idx[reject_with_mistake]
          
				if (len(expandPoints) > 0) and (len(currentPoints) > 0):
					if len(currentPoints) <= max_points:
						stacked_points.append(currentPoints)
						stacked_count.append(len(currentPoints))
						stacked_remove.extend(rejectClass)
						stacked_motion.extend(rejectClass_motion)
					else:
						subset = np.random.choice(len(currentPoints), max_points, replace=False)
						stacked_points.append(currentPoints[subset])
						stacked_count.append(max_points)
						rejectClass = rejectClass[subset]
						stacked_remove.extend(rejectClass)
						rejectClass_motion = rejectClass_motion[subset]
						stacked_motion.extend(rejectClass_motion)
					if len(expandPoints) <= max_points:
						stacked_neighbor_points.append(np.array(expandPoints))
						stacked_neighbor_count.append(len(expandPoints))
						stacked_add.extend(expandClass)
						stacked_motion.extend(expandClass_motion)
					else:
						subset = np.random.choice(len(expandPoints), max_points, replace=False)
						stacked_neighbor_points.append(expandPoints[subset])
						stacked_neighbor_count.append(max_points)
						expandClass = expandClass[subset]
						stacked_add.extend(expandClass)
						expandClass_motion = expandClass_motion[subset]
						stacked_motion.extend(expandClass_motion)
					iou = 1.0 * np.sum(np.logical_and(currentMask, gt_mask)) / np.sum(np.logical_or(currentMask, gt_mask))
					stacked_complete.append(iou)
#					stacked_complete.append(np.logical_not(np.logical_or(np.any(rejectClass), np.any(expandClass))))
					steps += 1
					add_mistake_prob = max(add_mistake_prob-0.01, 0.00)
					remove_mistake_prob = max(remove_mistake_prob-0.01, 0.00)

				if np.all(currentMask == gt_mask): #completed
					visited[currentMask] = True
					stacked_steps.append(steps)
					print('AREA %s room %d target %d : %d steps %d/%d (%.2f/%.2f IOU)'%(str(AREA), room_id, target_id, steps, np.sum(currentMask), np.sum(gt_mask), iou, originalScore))
					frm_done = True
				else:
					if steps < 500 and (np.any(expandClass) or np.any(rejectClass)): #continue growing
						#has matching neighbors: expand in those directions
						#update current mask
						currentMask[expandID] = True
						if len(rejectID) < len(mask_idx):
							currentMask[rejectID] = False
						if np.sum(currentMask)>0:
							nextMinDims = point_voxels[currentMask, :].min(axis=0)
							nextMaxDims = point_voxels[currentMask, :].max(axis=0)
						if not np.any(nextMinDims<minDims) and not np.any(nextMaxDims>maxDims):
							stuck = True
							iou = 1.0 * np.sum(np.logical_and(currentMask, gt_mask)) / np.sum(np.logical_or(currentMask, gt_mask))
							minDims = nextMinDims
							maxDims = nextMaxDims
					else: #no matching neighbors (early termination)
						if np.sum(currentMask) > cluster_threshold:
							visited[currentMask] = True
							stacked_steps.append(steps)
							print('AREA %s room %d target %d : %d steps %d/%d (%.2f/%.2f IOU)'%(str(AREA), room_id, target_id, steps, np.sum(currentMask), np.sum(gt_mask), iou, originalScore))
						frm_done = True

	for i in range(len(stacked_points)):
		center = np.median(stacked_points[i][:,:2], axis=0)
		feature_center = np.median(stacked_points[i][:,6:], axis=0)
		stacked_points[i][:,:2] -= center
		stacked_points[i][:,6:] -= feature_center
		if len(stacked_neighbor_points[i]) > 0:
			stacked_neighbor_points[i][:,:2] -= center
			stacked_neighbor_points[i][:,6:] -= feature_center

	#h5_fout = h5py.File('Z:\PHD\pcseg\multiseed\seed%d_area%s.h5'%(SEED,AREA),'w')
	h5_fout = h5py.File('Z:\PHD\pcseg\multiseed_res0p3\seed%d_area%s.h5'%(SEED,AREA),'w')
	h5_fout.create_dataset( 'points', data=np.vstack(stacked_points), compression='gzip', compression_opts=4, dtype=np.float32)
	h5_fout.create_dataset( 'count', data=stacked_count, compression='gzip', compression_opts=4, dtype=np.int32)
	h5_fout.create_dataset( 'neighbor_points', data=np.vstack(stacked_neighbor_points), compression='gzip', compression_opts=4, dtype=np.float32)
	h5_fout.create_dataset( 'neighbor_count', data=stacked_neighbor_count, compression='gzip', compression_opts=4, dtype=np.int32)
	h5_fout.create_dataset( 'add', data=stacked_add, compression='gzip', compression_opts=4, dtype=np.int32)
	h5_fout.create_dataset( 'remove', data=stacked_remove, compression='gzip', compression_opts=4, dtype=np.int32)
	h5_fout.create_dataset( 'motion', data=stacked_motion, compression='gzip', compression_opts=4, dtype=np.int32)
	h5_fout.create_dataset( 'steps', data=stacked_steps, compression='gzip', compression_opts=4, dtype=np.int32)
	h5_fout.create_dataset( 'complete', data=stacked_complete, compression='gzip', compression_opts=4, dtype=np.float32)
	h5_fout.close()

In [ ]:
# TRAIN ON SEMANTIC KITTI MODIFIED

import sys
import time

MAX_DATA = 600000

BATCH_SIZE = 150
NUM_INLIER_POINT = 512
NUM_NEIGHBOR_POINT = 512
MAX_EPOCH = 50
VAL_STEP = 5
TRAIN_AREA = [1,4,6,7]
VAL_AREA = [3]
FEATURE_SIZE = 11 #13
MULTISEED = 2
seeds=[3]
LITE = 1
initialized = False
cross_domain = False
numpy.random.seed(0)
numpy.set_printoptions(2,linewidth=100,suppress=True,sign=' ')
# for i in range(len(sys.argv)):
# 	if sys.argv[i]=='--train-area':
# 		TRAIN_AREA = sys.argv[i+1].split(',')
# 	if sys.argv[i]=='--val-area':
# 		VAL_AREA = sys.argv[i+1].split(',')
# 	if sys.argv[i]=='--cross-domain':
# 		cross_domain = True
# 	if sys.argv[i]=='--multiseed':
# 		MULTISEED = int(sys.argv[i+1])
# 	if sys.argv[i]=='--lite':
# 		LITE = int(sys.argv[i+1])

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = False
sess = tf.compat.v1.Session(config=config)
net = LrgNet(BATCH_SIZE, 1, NUM_INLIER_POINT, NUM_NEIGHBOR_POINT, FEATURE_SIZE, LITE)
saver = tf.compat.v1.train.Saver()

#MODEL_PATH = 'Z:/PHD/pcseg/models4/lrgnet_model%s_xyz_mod.ckpt'%VAL_AREA[0]
MODEL_PATH = 'Z:/PHD/pcseg/models4/lrgnet_model_xyz_mod_T1467_V3_600K.ckpt'

# if cross_domain:
# 	MODEL_PATH = 'models/cross_domain/lrgnet_%s.ckpt'%TRAIN_AREA[0]
# elif FEATURE_SIZE==6:
# 	MODEL_PATH = 'models/lrgnet_model%s_xyz.ckpt'%VAL_AREA[0]
# elif FEATURE_SIZE==9:
# 	MODEL_PATH = 'models/lrgnet_model%s_xyzrgb.ckpt'%VAL_AREA[0]
# elif FEATURE_SIZE==12:
# 	MODEL_PATH = 'models/lrgnet_model%s_xyzrgbn.ckpt'%VAL_AREA[0]
# else:
# 	# use full set of features
# 	if NUM_INLIER_POINT!=512 or NUM_NEIGHBOR_POINT!=512:
# 		MODEL_PATH = 'models/lrgnet_model%s_i_%d_j_%d.ckpt'%(VAL_AREA[0], NUM_INLIER_POINT, NUM_NEIGHBOR_POINT)
# 	elif LITE is not None:
# 		MODEL_PATH = 'models/lrgnet_model%s_lite_%d.ckpt'%(VAL_AREA[0], LITE)
# 	else:
# 		MODEL_PATH = 'models/lrgnet_model%s.ckpt'%VAL_AREA[0]
epoch_time = []

init = tf.compat.v1.global_variables_initializer()
sess.run(init, {})
for epoch in range(MAX_EPOCH):

	if not initialized or MULTISEED > 1:
		initialized = True
		train_inlier_points, train_inlier_count, train_neighbor_points, train_neighbor_count, train_add, train_remove, train_motion = [], [], [], [], [], [], []
		val_inlier_points, val_inlier_count, val_neighbor_points, val_neighbor_count, val_add, val_remove, val_motion = [], [], [], [], [], [], []

# 		if VAL_AREA is not None and (MULTISEED==0 and epoch % VAL_STEP == 0 or MULTISEED>0 and epoch % VAL_STEP == VAL_STEP-1):
# 			AREA_LIST = TRAIN_AREA + VAL_AREA
# 		else:
# 			AREA_LIST = TRAIN_AREA
		AREA_LIST = TRAIN_AREA + VAL_AREA
		for AREA in AREA_LIST:
# 			if isinstance(AREA, str) and AREA.startswith('synthetic'):
# 				f = h5py.File('data/staged_%s.h5' % AREA, 'r')
# 			elif MULTISEED > 0 and AREA in TRAIN_AREA:
# 				SEED = epoch % MULTISEED
# 				try:
# 					f = h5py.File('data/multiseed/seed%d_area%s.h5'%(SEED,AREA),'r')
# 				except OSError:
# 					continue
# 			else:
# 				f = h5py.File('data/staged_area%s.h5'%(AREA),'r')
			f = h5py.File('Z:/PHD/pcseg/multiseed/seed%d_areaKITTI_I20.h5'%(AREA), 'r')
			print('Loading %s ...'%f.filename)
			if VAL_AREA is not None and AREA in VAL_AREA:
				count = f['count'][:MAX_DATA]
				val_inlier_count.extend(count)
				points = f['points'][:]
				remove = f['remove'][:]
				idp = 0
				for i in range(len(count)):
					val_inlier_points.append(points[idp:idp+count[i], :FEATURE_SIZE])
					val_remove.append(remove[idp:idp+count[i]])
					idp += count[i]
				neighbor_count = f['neighbor_count'][:MAX_DATA]
				val_neighbor_count.extend(neighbor_count)
				neighbor_points = f['neighbor_points'][:]
				add = f['add'][:]
				idp = 0
				for i in range(len(neighbor_count)):
					val_neighbor_points.append(neighbor_points[idp:idp+neighbor_count[i], :FEATURE_SIZE])
					val_add.append(add[idp:idp+neighbor_count[i]])
					idp += neighbor_count[i]
				motion = f['motion'][:]
				idp = 0
				for i in range(len(count)):
					val_motion.append(motion[idp:(idp+neighbor_count[i]+count[i])])
					idp += count[i] + neighbor_count[i]
			if AREA in TRAIN_AREA:
				count = f['count'][:MAX_DATA]
				train_inlier_count.extend(count)
				points = f['points'][:]
				remove = f['remove'][:]
				idp = 0
				for i in range(len(count)):
					train_inlier_points.append(points[idp:idp+count[i], :FEATURE_SIZE])
					train_remove.append(remove[idp:idp+count[i]])
					idp += count[i]
				neighbor_count = f['neighbor_count'][:MAX_DATA]
				train_neighbor_count.extend(neighbor_count)
				neighbor_points = f['neighbor_points'][:]
				add = f['add'][:]
				idp = 0
				for i in range(len(neighbor_count)):
					train_neighbor_points.append(neighbor_points[idp:idp+neighbor_count[i], :FEATURE_SIZE])
					train_add.append(add[idp:idp+neighbor_count[i]])
					idp += neighbor_count[i]
				motion = f['motion'][:]
				idp = 0
				for i in range(len(count)):
					train_motion.append(motion[idp:(idp+neighbor_count[i]+count[i])])
					idp += count[i] + neighbor_count[i]
			if FEATURE_SIZE is None: 
				FEATURE_SIZE = points.shape[1]
			f.close()

		#filter out instances where the neighbor array is empty
		train_inlier_points = [train_inlier_points[i] for i in range(len(train_neighbor_count)) if train_neighbor_count[i]>0]
		train_inlier_count = [train_inlier_count[i] for i in range(len(train_neighbor_count)) if train_neighbor_count[i]>0]
		train_neighbor_points = [train_neighbor_points[i] for i in range(len(train_neighbor_count)) if train_neighbor_count[i]>0]
		train_add = [train_add[i] for i in range(len(train_neighbor_count)) if train_neighbor_count[i]>0]
		train_remove = [train_remove[i] for i in range(len(train_neighbor_count)) if train_neighbor_count[i]>0]
		train_neighbor_count = [train_neighbor_count[i] for i in range(len(train_neighbor_count)) if train_neighbor_count[i]>0]
		train_motion = [train_motion[i] for i in range(len(train_neighbor_count)) if train_neighbor_count[i]>0]
		val_inlier_points = [val_inlier_points[i] for i in range(len(val_neighbor_count)) if val_neighbor_count[i]>0]
		val_inlier_count = [val_inlier_count[i] for i in range(len(val_neighbor_count)) if val_neighbor_count[i]>0]
		val_neighbor_points = [val_neighbor_points[i] for i in range(len(val_neighbor_count)) if val_neighbor_count[i]>0]
		val_add = [val_add[i] for i in range(len(val_neighbor_count)) if val_neighbor_count[i]>0]
		val_remove = [val_remove[i] for i in range(len(val_neighbor_count)) if val_neighbor_count[i]>0]
		val_neighbor_count = [val_neighbor_count[i] for i in range(len(val_neighbor_count)) if val_neighbor_count[i]>0]
		val_motion = [val_motion[i] for i in range(len(val_neighbor_count)) if val_neighbor_count[i]>0]
		if len(train_inlier_points)==0:
			continue
		print('train',len(train_inlier_points),train_inlier_points[0].shape, len(train_neighbor_points))
		print('val',len(val_inlier_points), len(val_neighbor_points))

	idx = numpy.arange(len(train_inlier_points))
	numpy.random.shuffle(idx)
	inlier_points = numpy.zeros((BATCH_SIZE, NUM_INLIER_POINT, FEATURE_SIZE))
	neighbor_points = numpy.zeros((BATCH_SIZE, NUM_NEIGHBOR_POINT, FEATURE_SIZE))
	input_add = numpy.zeros((BATCH_SIZE, NUM_NEIGHBOR_POINT), dtype=numpy.int32)
	input_remove = numpy.zeros((BATCH_SIZE, NUM_INLIER_POINT), dtype=numpy.int32)
	input_motion = numpy.zeros((BATCH_SIZE, NUM_INLIER_POINT+NUM_NEIGHBOR_POINT), dtype=numpy.int32)

	loss_arr = []
	add_prc_arr = []
	add_rcl_arr = []
	rmv_prc_arr = []
	rmv_rcl_arr = []
	mot_prc_arr = []
	mot_rcl_arr = []
	num_batches = int(len(train_inlier_points) / BATCH_SIZE)
	start_time = time.time()
	for batch_id in range(num_batches):
		start_idx = batch_id * BATCH_SIZE
		end_idx = (batch_id + 1) * BATCH_SIZE
		for i in range(BATCH_SIZE):
			points_idx = idx[start_idx+i]
			N = train_inlier_count[points_idx]
			if N >= NUM_INLIER_POINT:
				subset = numpy.random.choice(N, NUM_INLIER_POINT, replace=False)
			else:
				subset = list(range(N)) + list(numpy.random.choice(N, NUM_INLIER_POINT-N, replace=True))
			inlier_points[i,:,:] = train_inlier_points[points_idx][subset, :]
			input_remove[i,:] = train_remove[points_idx][subset]
			N = train_neighbor_count[points_idx]
			if N >= NUM_NEIGHBOR_POINT:
				subset = numpy.random.choice(N, NUM_NEIGHBOR_POINT, replace=False)
			else:
				subset = list(range(N)) + list(numpy.random.choice(N, NUM_NEIGHBOR_POINT-N, replace=True))
			neighbor_points[i,:,:] = train_neighbor_points[points_idx][subset, :]
			input_add[i,:] = train_add[points_idx][subset]
			N = train_neighbor_count[points_idx] + train_inlier_count[points_idx]
			if N >= (NUM_NEIGHBOR_POINT+NUM_INLIER_POINT):
				subset = numpy.random.choice(N, NUM_NEIGHBOR_POINT+NUM_INLIER_POINT, replace=False)
			else:
				subset = list(range(N)) + list(numpy.random.choice(N, NUM_NEIGHBOR_POINT+NUM_INLIER_POINT-N, replace=True))
			input_motion[i,:] = train_motion[points_idx][subset]
		_, ls, ap, ar, rp, rr, mp, mr = sess.run([net.train_op, net.loss, net.add_prc, net.add_rcl, net.remove_prc, net.remove_rcl, net.motion_prc, net.motion_rcl],
			{net.inlier_pl:inlier_points, net.neighbor_pl:neighbor_points, net.add_mask_pl:input_add, net.remove_mask_pl:input_remove, net.motion_mask_pl:input_motion})
		loss_arr.append(ls)
		add_prc_arr.append(ap)
		add_rcl_arr.append(ar)
		rmv_prc_arr.append(rp)
		rmv_rcl_arr.append(rr)
		mot_prc_arr.append(mp)
		mot_rcl_arr.append(mr)
	epoch_time.append(time.time() - start_time)
	print("Epoch %d loss %.2f add %.2f/%.2f rmv %.2f/%.2f"%(epoch,numpy.mean(loss_arr),numpy.mean(add_prc_arr),numpy.mean(add_rcl_arr),numpy.mean(rmv_prc_arr), numpy.mean(rmv_rcl_arr)))

	if VAL_AREA is not None and epoch % VAL_STEP == VAL_STEP - 1:
		loss_arr = []
		add_prc_arr = []
		add_rcl_arr = []
		rmv_prc_arr = []
		rmv_rcl_arr = []
		mot_prc_arr = []
		mot_rcl_arr = []
		num_batches = int(len(val_inlier_points) / BATCH_SIZE)
		for batch_id in range(num_batches):
			start_idx = batch_id * BATCH_SIZE
			end_idx = (batch_id + 1) * BATCH_SIZE
			for i in range(BATCH_SIZE):
				points_idx = start_idx+i
				N = val_inlier_count[points_idx]
				if N >= NUM_INLIER_POINT:
					subset = numpy.random.choice(N, NUM_INLIER_POINT, replace=False)
				else:
					subset = list(range(N)) + list(numpy.random.choice(N, NUM_INLIER_POINT-N, replace=True))
				inlier_points[i,:,:] = val_inlier_points[points_idx][subset, :]
				input_remove[i,:] = val_remove[points_idx][subset]
				N = val_neighbor_count[points_idx]
				if N >= NUM_INLIER_POINT:
					subset = numpy.random.choice(N, NUM_NEIGHBOR_POINT, replace=False)
				else:
					subset = list(range(N)) + list(numpy.random.choice(N, NUM_NEIGHBOR_POINT-N, replace=True))
				neighbor_points[i,:,:] = val_neighbor_points[points_idx][subset, :]
				input_add[i,:] = val_add[points_idx][subset]
				N = val_neighbor_count[points_idx] + val_inlier_count[points_idx]
				if N >= (NUM_NEIGHBOR_POINT+NUM_INLIER_POINT):
					subset = numpy.random.choice(N, NUM_NEIGHBOR_POINT+NUM_INLIER_POINT, replace=False)
				else:
					subset = list(range(N)) + list(numpy.random.choice(N, NUM_NEIGHBOR_POINT+NUM_INLIER_POINT-N, replace=True))
				input_motion[i,:] = val_motion[points_idx][subset]
			ls, ap, ar, rp, rr, mp, mr = sess.run([net.loss, net.add_prc, net.add_rcl, net.remove_prc, net.remove_rcl, net.motion_prc, net.motion_rcl],
				{net.inlier_pl:inlier_points, net.neighbor_pl:neighbor_points, net.add_mask_pl:input_add, net.remove_mask_pl:input_remove, net.motion_mask_pl:input_motion})
			loss_arr.append(ls)
			add_prc_arr.append(ap)
			add_rcl_arr.append(ar)
			rmv_prc_arr.append(rp)
			rmv_rcl_arr.append(rr)
			mot_prc_arr.append(mp)
			mot_rcl_arr.append(mr)
		print("Validation %d loss %.2f add %.2f/%.2f rmv %.2f/%.2f"%(epoch,numpy.mean(loss_arr),numpy.mean(add_prc_arr),numpy.mean(add_rcl_arr),numpy.mean(rmv_prc_arr), numpy.mean(rmv_rcl_arr)))
	saver.save(sess, MODEL_PATH)
print("Avg Epoch Time: %.3f" % numpy.mean(epoch_time))
# print("GPU Mem: " , tf.config.experimental.get_memory_info('GPU:0'))
#saver.save(sess, MODEL_PATH)

In [ ]:
# DATA EXPLORATION PRE_TRAINING

import numpy as np
import pandas as pd
import h5py
import os
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import matplotlib.animation as animation

FEATURE_SIZE = 11

f = h5py.File('Z:/PHD/pcseg/multiseed/seed3_areaKITTI_I20.h5', 'r')

points = f['points'][:]
count = f['count'][:]
remove = f['remove'][:]
add = f['add'][:]
motion = f['motion'][:]
neighbor_points = f['neighbor_points'][:]
neighbor_count = f['neighbor_count'][:]

train_inlier_points = []
train_remove = []
train_neighbor_points = []
train_add = []
train_motion = []

idp = 0
for i in range(len(count)):
    train_inlier_points.append(points[idp:idp+count[i], :FEATURE_SIZE])
    train_remove.append(remove[idp:idp+count[i]])
    idp += count[i]
    
idp = 0
for i in range(len(neighbor_count)):
    train_neighbor_points.append(neighbor_points[idp:idp+neighbor_count[i], :FEATURE_SIZE])
    train_add.append(add[idp:idp+neighbor_count[i]])
    idp += neighbor_count[i]
    
idp = 0
for i in range(len(count)):
    train_motion.append(motion[idp:idp+count[i]+neighbor_count[i]])
    idp += count[i] + neighbor_count[i]
    
step_index = 435

# count_series = pd.Series(count)#[count <10])
# count_series.plot.hist(grid=True, bins=20, rwidth=0.9,color='#607c8e')

# neighbor_count_series = pd.Series(neighbor_count)#[count <10])
# neighbor_count_series.plot.hist(grid=True, bins=20, rwidth=0.9,color='#607c8e')

print(len(points))
print(len(neighbor_points))
print(len(count))
print(len(neighbor_count))
print(len(add))
print(len(remove))
print(len(motion))
print(len(train_inlier_points))
print(len(train_neighbor_points))

frac_add = []
frac_rem = []
frac_mot = []

for i in range(len(count)):
    frac_add.append(float(sum(train_add[i]))/float(len(train_add[i])))
    frac_rem.append(float(sum(train_remove[i]))/float(len(train_remove[i])))
    frac_mot.append(float(sum(train_motion[i]))/float(len(train_motion[i])))
    
# add_series = pd.Series(frac_add)#[count <10])
# add_series.plot.hist(grid=True, bins=20, rwidth=0.9,color='#607c8e')

rem_series = pd.Series(frac_rem)#[count <10])
rem_series.plot.hist(grid=True, bins=20, rwidth=0.9,color='#607c8e')

# mot_series = pd.Series(frac_mot)#[count <10])
# mot_series.plot.hist(grid=True, bins=20, rwidth=0.9,color='#607c8e')



    
# fig1,ax1 = plt.subplots(figsize=(10, 10))
# rot1=45
# rot2=45
# rot3=0
# r = R.from_euler('zyx', [rot1, rot2, rot3], degrees=True)
# scale = 50

# draw_points=train_inlier_points[step_index][:,:3]
# npt=len(draw_points)
# rgb=np.zeros((1000,1000,3),dtype=np.uint8)
# for k in range(npt):
#     vr=r.apply(draw_points[k,0:3])
    
#     x=int(vr[0]*scale)+500
#     y=int(vr[1]*scale)+500
#     if (x>=0) and (x<1000) and (y>=0) and (y<1000):
#         rgb[x,y,0]=255
#         rgb[x,y,1]=255
#         rgb[x,y,2]=0
        
# draw_points=train_neighbor_points[step_index][:,:FEATURE_SIZE]
# npt=len(draw_points)
# for k in range(npt):
#     vr=r.apply(draw_points[k,0:3])
    
#     x=int(vr[0]*scale)+500
#     y=int(vr[1]*scale)+500
#     if (x>=0) and (x<1000) and (y>=0) and (y<1000):
#         rgb[x,y,0]=0
#         rgb[x,y,1]=255
#         rgb[x,y,2]=255
            
# im = ax1.imshow(rgb)
# plt.show()

In [2]:
# INFER SEGMENTATION ON A WHOLE KITTI SEQUENCE

import motrackers
import tensorflow as tf
import h5py
import itertools
import time
import scipy
#from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, adjusted_mutual_info_score

classes_kitti = [''] * 260
classes_kitti[0] = "unlabeled"
classes_kitti[1] = "outlier"
classes_kitti[10] = "car"
classes_kitti[11] = "bicycle"
classes_kitti[13] = "bus"
classes_kitti[15] = "motorcycle"
classes_kitti[16] = "on-rails"
classes_kitti[18] = "truck"
classes_kitti[20] = "other-vehicle"
classes_kitti[30] = "person"
classes_kitti[31] = "bicyclist"
classes_kitti[32] = "motorcyclist"
classes_kitti[40] = "road"
classes_kitti[44] = "parking"
classes_kitti[48] = "sidewalk"
classes_kitti[49] = "other-ground"
classes_kitti[50] = "building"
classes_kitti[51] = "fence"
classes_kitti[52] = "other-structure"
classes_kitti[60] = "lane-marking"
classes_kitti[70] = "vegetation"
classes_kitti[71] = "trunk"
classes_kitti[72] = "terrain"
classes_kitti[80] = "pole"
classes_kitti[81] = "traffic-sign"
classes_kitti[99] = "other-object"
classes_kitti[252] = "moving-car"
classes_kitti[253] = "moving-bicyclist"
classes_kitti[254] = "moving-person"
classes_kitti[255] = "moving-motorcyclist"
classes_kitti[256] = "moving-on-rails"
classes_kitti[257] = "moving-bus"
classes_kitti[258] = "moving-truck"
classes_kitti[259] = "moving-other-vehicle"

agg_nmi = []
agg_ami = []
agg_ars = []
agg_prc = []
agg_rcl = []
agg_iou = []
comp_time_analysis = {
	'feature': [],
	'net': [],
	'neighbor': [],
	'inlier': [],
	'current_net' : [],
	'current_neighbor' : [],
	'current_inlier' : [],
	'iter_net' : [],
	'iter_neighbor' : [],
	'iter_inlier' : [],
}

def saveCSV(filename, points):
	f = open(filename,'w')
	for p in points:
		f.write("%f,%f,%f,%d,%d,%d\n"%(p[0],p[1],p[2],p[3],p[4],p[5]))
	f.close()
	print('Saved to %s: (%d points)'%(filename, len(points)))
    
def saveCSVM(filename, points):
	f = open(filename,'w')
	for p in points:
		f.write("%f,%f,%f,%d,%d,%d,%d,%f,%f,%f,%d\n"%(p[0],p[1],p[2],p[3],p[4],p[5],p[6],p[7],p[8],p[9],p[10]))
	f.close()
	print('Saved to %s: (%d points)'%(filename, len(points)))


def loadFromH5ModV2(filename):
	f = h5py.File(filename,'r')
	all_points = f['points'][:]
	count_room = f['count_room'][:]
	tmp_points = []
	idp = 0
	for i in range(len(count_room)):
		tmp_points.append(all_points[idp:idp+count_room[i], :])
		idp += count_room[i]
	f.close()
	room = []
	labels = []
	class_labels = []
	frame_labels = []
	motion_labels = []
	for i in range(len(tmp_points)):
		room.append(tmp_points[i][:,:-5])
		labels.append(tmp_points[i][:,-5].astype(int))
		class_labels.append(tmp_points[i][:,-4].astype(int))
		frame_labels.append(tmp_points[i][:,-3].astype(int))
		motion_labels.append(tmp_points[i][:,-1].astype(int))
	return room, labels, class_labels, frame_labels, motion_labels

sequence = '04'
series = 'MOT_600K_4S_TR_1_LITE_04_0p3'

#all_points,all_obj_id,all_cls_id,all_frm_id,all_motion = loadFromH5ModV2('Z:\PHD\pcseg\KITTI_MOD\KITTI_I20_06.h5')
# all_points,all_obj_id,all_cls_id,all_frm_id,all_motion = loadFromH5ModV2('Z:\PHD\pcseg\KITTI_MOD\KITTI_I20_01.h5')
all_points,all_obj_id,all_cls_id,all_frm_id,all_motion = loadFromH5ModV2('Z:\PHD\pcseg\KITTI_MOD\KITTI_I20_'+sequence+'.h5')

NUM_INLIER_POINT = 512
NUM_NEIGHBOR_POINT = 512
TRAIN_AREA = [1]
VAL_AREA = [3]
FEATURE_SIZE = 11
LITE = 1
resolution = 0.3
add_threshold = 0.5
rmv_threshold = 0.5
cluster_threshold = 10
save_results = True
save_id = 0

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = False
sess = tf.compat.v1.Session(config=config)

net = LrgNet(1, 1, NUM_INLIER_POINT, NUM_NEIGHBOR_POINT, FEATURE_SIZE, LITE)

#MODEL_PATH = 'Z:/PHD/pcseg/models4/lrgnet_model%s_xyz_mod.ckpt'%VAL_AREA[0]

MODEL_PATH = 'Z:/PHD/pcseg/models4/lrgnet_model_xyz_mod_T1467_V3_600K.ckpt'

saver = tf.compat.v1.train.Saver()
saver.restore(sess, MODEL_PATH)
print('Restored from %s'%MODEL_PATH)

file=open('Z:/PHD/pcseg/results/lrg/'+series+'/'+series+'.csv','w')

for room_id in range(len(all_points)):
	print('Room: ',room_id)
	unequalized_points = all_points[room_id]
	obj_id = all_obj_id[room_id]
	cls_id = all_cls_id[room_id]
	frm_id = all_frm_id[room_id]
	motion = all_motion[room_id]

	#equalize resolution
	t1 = time.time()
	equalized_idx = []
	unequalized_idx = []
	equalized_map = {}
	normal_grid = {}
	for i in range(len(unequalized_points)):
		k = tuple(numpy.round(unequalized_points[i,:3]/resolution).astype(int))
		if not k in equalized_map:
			equalized_map[k] = len(equalized_idx)
			equalized_idx.append(i)
		unequalized_idx.append(equalized_map[k])
		if not k in normal_grid:
			normal_grid[k] = []
		normal_grid[k].append(i)
	points = unequalized_points[equalized_idx]
	obj_id = obj_id[equalized_idx]
	cls_id = cls_id[equalized_idx]
	frm_id = frm_id[equalized_idx]
	motion = motion[equalized_idx]
	xyz = points[:,:3]
	room_coordinates = (xyz - xyz.min(axis=0)) / (xyz.max(axis=0) - xyz.min(axis=0))

	#compute normals
	normals = []
	curvatures = []
	for i in range(len(points)):
		k = tuple(numpy.round(points[i,:3]/resolution).astype(int))
		neighbors = []
		for offset in itertools.product([-1,0,1],[-1,0,1],[-1,0,1]):
			kk = (k[0]+offset[0], k[1]+offset[1], k[2]+offset[2])
			if kk in normal_grid:
				neighbors.extend(normal_grid[kk])
		accA = numpy.zeros((3,3))
		accB = numpy.zeros(3)
		for n in neighbors:
			p = unequalized_points[n,:3]
			accA += numpy.outer(p,p)
			accB += p
		cov = accA / len(neighbors) - numpy.outer(accB, accB) / len(neighbors)**2
		U,S,V = numpy.linalg.svd(cov)
		normals.append(numpy.fabs(V[2]))
		curvature = S[2] / (S[0] + S[1] + S[2])
		curvatures.append(numpy.fabs(curvature))
	curvatures = numpy.array(curvatures)
	curvatures = curvatures/curvatures.max()
	normals = numpy.array(normals)
	comp_time_analysis['feature'].append(time.time() - t1)
    
	points = numpy.hstack((xyz, room_coordinates, normals, curvatures.reshape(-1,1), frm_id.reshape(-1,1))).astype(numpy.float32)

	point_voxels = numpy.round(points[:,:3]/resolution).astype(int)
	cluster_label = numpy.zeros(len(points), dtype=int)
	motion_label = numpy.zeros(len(points), dtype=int)
	cluster_id = 1
	visited = numpy.zeros(len(point_voxels), dtype=bool)
	inlier_points = numpy.zeros((1, NUM_INLIER_POINT, FEATURE_SIZE), dtype=numpy.float32)
	neighbor_points = numpy.zeros((1, NUM_NEIGHBOR_POINT, FEATURE_SIZE), dtype=numpy.float32)
	input_add = numpy.zeros((1, NUM_NEIGHBOR_POINT), dtype=numpy.int32)
	input_remove = numpy.zeros((1, NUM_INLIER_POINT), dtype=numpy.int32)
	input_motion = numpy.zeros((1, NUM_INLIER_POINT+NUM_NEIGHBOR_POINT), dtype=numpy.int32)
	order = numpy.argsort(curvatures)
	#iterate over each object in the room
	#for seed_id in range(len(point_voxels)):
	for seed_id in numpy.arange(len(points))[order]:
		if visited[seed_id]:
			continue
		seed_voxel = point_voxels[seed_id]
		target_id = obj_id[seed_id]
		target_class = classes_kitti[cls_id[numpy.nonzero(obj_id==target_id)[0][0]]]
		gt_mask = obj_id==target_id
		obj_voxels = point_voxels[gt_mask]
		obj_voxel_set = set([tuple(p) for p in obj_voxels])
		original_minDims = obj_voxels.min(axis=0)
		original_maxDims = obj_voxels.max(axis=0)
		currentMask = numpy.zeros(len(points), dtype=bool)
		currentMask[seed_id] = True
		minDims = seed_voxel.copy()
		maxDims = seed_voxel.copy()
		seqMinDims = minDims
		seqMaxDims = maxDims
		steps = 0
		stuck = 0
		maskLogProb = 0

		#perform region growing
		while True:

			def stop_growing(reason):
				global cluster_id, start_time
				visited[currentMask] = True
				if numpy.sum(currentMask) > cluster_threshold:
					cluster_label[currentMask] = cluster_id
					cluster_id += 1
					iou = 1.0 * numpy.sum(numpy.logical_and(gt_mask,currentMask)) / numpy.sum(numpy.logical_or(gt_mask,currentMask))
					#print('room %d target %3d %.4s: step %3d %4d/%4d points IOU %.3f add %.3f rmv %.3f %s'%(room_id, target_id, target_class, steps, numpy.sum(currentMask), numpy.sum(gt_mask), iou, add_acc, rmv_acc, reason))
					strout='room %d target %3d %.4s: step %3d %4d/%4d points IOU %.3f add %.3f rmv %.3f mot %.3f %s'%(room_id, target_id, target_class, steps, numpy.sum(currentMask), numpy.sum(gt_mask), iou, add_acc, rmv_acc, mot_acc, reason)
					strfil='%d,%3d,%.4s,%3d,%4d,%4d,%.3f,%.3f,%.3f,%.3f,%s'%(room_id, target_id, target_class, steps, numpy.sum(currentMask), numpy.sum(gt_mask), iou, add_acc, rmv_acc, mot_acc, reason)
					print(strout)
					file.write(strfil)
					file.write('\n')
                    
			#determine the current points and the neighboring points
			t = time.time()
			currentPoints = points[currentMask, :].copy()
			newMinDims = minDims.copy()	
			newMaxDims = maxDims.copy()	
			newMinDims -= 1
			newMaxDims += 1
			mask = numpy.logical_and(numpy.all(point_voxels>=newMinDims,axis=1), numpy.all(point_voxels<=newMaxDims, axis=1))
			mask = numpy.logical_and(mask, numpy.logical_not(currentMask))
			mask = numpy.logical_and(mask, numpy.logical_not(visited))
			expandPoints = points[mask, :].copy()
			expandClass = obj_id[mask] == target_id
			expandClass_motion = motion[mask]
			rejectClass = obj_id[currentMask] != target_id
			rejectClass_motion = motion[currentMask] 

			if len(expandPoints)==0: #no neighbors (early termination)
				stop_growing('noneighbor')
				break

			if len(currentPoints) >= NUM_INLIER_POINT:
				subset = numpy.random.choice(len(currentPoints), NUM_INLIER_POINT, replace=False)
			else:
				subset = list(range(len(currentPoints))) + list(numpy.random.choice(len(currentPoints), NUM_INLIER_POINT-len(currentPoints), replace=True))
			center = numpy.median(currentPoints, axis=0)
			expandPoints = numpy.array(expandPoints)
			expandPoints[:,:2] -= center[:2]
			expandPoints[:,6:] -= center[6:]
			inlier_points[0,:,:] = currentPoints[subset, :]
			inlier_points[0,:,:2] -= center[:2]
			inlier_points[0,:,6:] -= center[6:]
			input_remove[0,:] = numpy.array(rejectClass)[subset]
            
			input_motion[0,:NUM_INLIER_POINT]=numpy.array(rejectClass_motion)[subset]            
            
			if len(expandPoints) >= NUM_NEIGHBOR_POINT:
				subset = numpy.random.choice(len(expandPoints), NUM_NEIGHBOR_POINT, replace=False)
			else:
				subset = list(range(len(expandPoints))) + list(numpy.random.choice(len(expandPoints), NUM_NEIGHBOR_POINT-len(expandPoints), replace=True))
			neighbor_points[0,:,:] = numpy.array(expandPoints)[subset, :]
			input_add[0,:] = numpy.array(expandClass)[subset]
            
			input_motion[0,NUM_INLIER_POINT:]=numpy.array(expandClass_motion)[subset]            
            
			comp_time_analysis['current_neighbor'].append(time.time() - t)
			t = time.time()
			ls, add,add_acc, rmv,rmv_acc, mot,mot_acc = sess.run([net.loss, net.add_output, net.add_acc, net.remove_output, net.remove_acc, net.motion_output, net.motion_acc],
				{net.inlier_pl:inlier_points, net.neighbor_pl:neighbor_points, net.add_mask_pl:input_add, net.remove_mask_pl:input_remove, net.motion_mask_pl:input_motion})
			comp_time_analysis['current_net'].append(time.time() - t)
			t = time.time()

			add_conf = scipy.special.softmax(add[0], axis=-1)[:,1]
			rmv_conf = scipy.special.softmax(rmv[0], axis=-1)[:,1]
			mot_conf = scipy.special.softmax(mot[0], axis=-1)[:,1]
#			add_mask = add_conf > add_threshold
#			rmv_mask = rmv_conf > rmv_threshold
			add_mask = numpy.random.random(len(add_conf)) < add_conf
			rmv_mask = numpy.random.random(len(rmv_conf)) < rmv_conf
			mot_mask = numpy.random.random(len(mot_conf)) < mot_conf
#			add_mask = input_add[0].astype(bool)
#			rmv_mask = input_remove[0].astype(bool)
			addPoints = neighbor_points[0,:,:][add_mask]
			addPoints[:,:2] += center[:2]
			addVoxels = numpy.round(addPoints[:,:3]/resolution).astype(int)
			addSet = set([tuple(p) for p in addVoxels])
			rmvPoints = inlier_points[0,:,:][rmv_mask]
			rmvPoints[:,:2] += center[:2]
			rmvVoxels = numpy.round(rmvPoints[:,:3]/resolution).astype(int)
			rmvSet = set([tuple(p) for p in rmvVoxels])

			motPoints = numpy.vstack((inlier_points[0,:,:],neighbor_points[0,:,:]))[mot_mask]
			motPoints[:,:2] += center[:2]
			motVoxels = numpy.round(motPoints[:,:3]/resolution).astype(int)
			motSet = set([tuple(p) for p in motVoxels])

			updated = False
			iou = 1.0 * numpy.sum(numpy.logical_and(gt_mask,currentMask)) / numpy.sum(numpy.logical_or(gt_mask,currentMask))
#			print('%d/%d points %d outliers %d/%d add %d/%d rmv %.2f iou'%(numpy.sum(numpy.logical_and(currentMask, gt_mask)), numpy.sum(gt_mask),
#				numpy.sum(numpy.logical_and(gt_mask==0, currentMask)), len(addSet), len(expandPoints), len(rmvSet), len(currentPoints), iou))
			for i in range(len(point_voxels)):
				if not currentMask[i] and tuple(point_voxels[i]) in addSet:
					currentMask[i] = True
					updated = True
				if tuple(point_voxels[i]) in rmvSet:
					currentMask[i] = False
				if tuple(point_voxels[i]) in motSet:                    
					motion_label[i] = 1
			steps += 1
			comp_time_analysis['current_inlier'].append(time.time() - t)

			if updated: #continue growing
				minDims = point_voxels[currentMask, :].min(axis=0)
				maxDims = point_voxels[currentMask, :].max(axis=0)
				if not numpy.any(minDims<seqMinDims) and not numpy.any(maxDims>seqMaxDims):
					if stuck >= 1:
						stop_growing('stuck')
						break
					else:
						stuck += 1
				else:
					stuck = 0
				seqMinDims = numpy.minimum(seqMinDims, minDims)
				seqMaxDims = numpy.maximum(seqMaxDims, maxDims)
			else: #no matching neighbors (early termination)
				stop_growing('noexpand')
				break

	#fill in points with no labels
	nonzero_idx = numpy.nonzero(cluster_label)[0]
	nonzero_points = points[nonzero_idx, :]
	filled_cluster_label = cluster_label.copy()
	for i in numpy.nonzero(cluster_label==0)[0]:
		d = numpy.sum((nonzero_points - points[i])**2, axis=1)
		closest_idx = numpy.argmin(d)
		filled_cluster_label[i] = cluster_label[nonzero_idx[closest_idx]]
	cluster_label = filled_cluster_label
	print('%d %d points: %.2fs' % (room_id, len(unequalized_points), time.time() - t1))

	#calculate statistics 
	gt_match = 0
	match_id = 0
	dt_match = numpy.zeros(cluster_label.max(), dtype=bool)
	cluster_label2 = numpy.zeros(len(cluster_label), dtype=int)
	room_iou = []
	unique_id, count = numpy.unique(obj_id, return_counts=True)
	for k in range(len(unique_id)):
		i = unique_id[numpy.argsort(count)][::-1][k]
		best_iou = 0
		for j in range(1, cluster_label.max()+1):
			if not dt_match[j-1]:
				iou = 1.0 * numpy.sum(numpy.logical_and(obj_id==i, cluster_label==j)) / numpy.sum(numpy.logical_or(obj_id==i, cluster_label==j))
				best_iou = max(best_iou, iou)
				if iou > 0.5:
					dt_match[j-1] = True
					gt_match += 1
					cluster_label2[cluster_label==j] = k+1
					break
		room_iou.append(best_iou)
	for j in range(1,cluster_label.max()+1):
		if not dt_match[j-1]:
			cluster_label2[cluster_label==j] = j + obj_id.max()
	prc = numpy.mean(dt_match)
	rcl = 1.0 * gt_match / len(set(obj_id))
	room_iou = numpy.mean(room_iou)

	nmi = 0#normalized_mutual_info_score(obj_id,cluster_label)
	ami = 0#adjusted_mutual_info_score(obj_id,cluster_label)
	ars = 0#adjusted_rand_score(obj_id,cluster_label)
	agg_nmi.append(nmi)
	agg_ami.append(ami)
	agg_ars.append(ars)
	agg_prc.append(prc)
	agg_rcl.append(rcl)
	agg_iou.append(room_iou)
	print("room %d NMI: %.2f AMI: %.2f ARS: %.2f PRC: %.2f RCL: %.2f IOU: %.2f"%(room_id, nmi,ami,ars, prc, rcl, room_iou))

	comp_time_analysis['neighbor'].append(sum(comp_time_analysis['current_neighbor']))
	comp_time_analysis['iter_neighbor'].extend(comp_time_analysis['current_neighbor'])
	comp_time_analysis['current_neighbor'] = []
	comp_time_analysis['net'].append(sum(comp_time_analysis['current_net']))
	comp_time_analysis['iter_net'].extend(comp_time_analysis['current_net'])
	comp_time_analysis['current_net'] = []
	comp_time_analysis['inlier'].append(sum(comp_time_analysis['current_inlier']))
	comp_time_analysis['iter_inlier'].extend(comp_time_analysis['current_inlier'])
	comp_time_analysis['current_inlier'] = []

	print(cluster_label2.shape)
	print(len(unequalized_idx))
	#save point cloud results to file
	if save_results:
		#color_sample_state = numpy.random.RandomState(0)
		#obj_color = color_sample_state.randint(0,255,(numpy.max(cluster_label2)+1,3))
		#obj_color[0] = [100,100,100]
		#unequalized_points[:,3:6] = obj_color[cluster_label2,:][unequalized_idx]
		colors=numpy.random.randint(0,255,(numpy.max(cluster_label)+1,3))
		colors[0] = [100,100,100]
		for i in range(len(points)):
			points[i,3:6] = colors[cluster_label[i],:]
			points[i,6] = motion_label[i]
		#saveCSV('Z:/PHD/pcseg/results/lrg/MOT/%d.csv'%save_id, points)
		saveCSVM('Z:/PHD/pcseg/results/lrg/'+series+'/M_%d.csv'%save_id, points)
		save_id += 1

print('NMI: %.2f+-%.2f AMI: %.2f+-%.2f ARS: %.2f+-%.2f PRC %.2f+-%.2f RCL %.2f+-%.2f IOU %.2f+-%.2f'%
(numpy.mean(agg_nmi), numpy.std(agg_nmi),numpy.mean(agg_ami),numpy.std(agg_ami),numpy.mean(agg_ars),numpy.std(agg_ars),
numpy.mean(agg_prc), numpy.std(agg_prc), numpy.mean(agg_rcl), numpy.std(agg_rcl), numpy.mean(agg_iou), numpy.std(agg_iou)))

file.close()

INFO:tensorflow:Restoring parameters from Z:/PHD/pcseg/models4/lrgnet_model_xyz_mod_T1467_V3_600K.ckpt
Restored from Z:/PHD/pcseg/models4/lrgnet_model_xyz_mod_T1467_V3_600K.ckpt
Room:  0
room 0 target 187 park: step 311 118815/ 240 points IOU 0.002 add 0.000 rmv 0.006 mot 0.971 noneighbor
room 0 target  43 unla: step  20  262/ 196 points IOU 0.735 add 0.227 rmv 0.730 mot 1.000 stuck
room 0 target 195 unla: step  45  228/ 212 points IOU 0.930 add 1.000 rmv 0.932 mot 1.000 noexpand
room 0 target 174 unla: step  16   38/  77 points IOU 0.494 add 1.000 rmv 0.379 mot 1.000 stuck
room 0 target  37 unla: step  31  936/1016 points IOU 0.804 add 0.080 rmv 0.932 mot 1.000 stuck
room 0 target  37 unla: step  19  145/1016 points IOU 0.143 add 1.000 rmv 1.000 mot 1.000 stuck
room 0 target 192 unla: step  26  144/ 144 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 0 target  92 unla: step  30  582/ 236 points IOU 0.405 add 0.000 rmv 0.391 mot 1.000 noneighbor
room 0 target 174 unla: s

room 4 target 104 unla: step  16  162/ 161 points IOU 0.994 add 1.000 rmv 0.990 mot 1.000 noneighbor
room 4 target 183 unla: step  12   55/  55 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 4 target 106 unla: step  27  187/ 204 points IOU 0.917 add 0.303 rmv 1.000 mot 1.000 stuck
room 4 target  41 unla: step  18  199/ 199 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 4 target 201 unla: step   8   57/  57 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 4 target 106 unla: step  12   15/ 204 points IOU 0.074 add 1.000 rmv 1.000 mot 1.000 stuck
room 4 target 191 unla: step  13   53/  55 points IOU 0.964 add 1.000 rmv 1.000 mot 1.000 stuck
4 122705 points: 319.63s
room 4 NMI: 0.00 AMI: 0.00 ARS: 0.00 PRC: 0.83 RCL: 0.05 IOU: 0.05
(122705,)
122705
Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_4.csv: (122705 points)
Room:  5
room 5 target 141 othe: step 293 121702/4959 points IOU 0.041 add 0.000 rmv 0.029 mot 0.977 noneighbor

room 10 target  94 unla: step  12   81/  81 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 10 target 137 unla: step  13  118/ 413 points IOU 0.286 add 0.469 rmv 0.125 mot 1.000 stuck
room 10 target  91 unla: step  11  126/ 159 points IOU 0.792 add 1.000 rmv 1.000 mot 1.000 stuck
room 10 target 137 unla: step  11   52/ 413 points IOU 0.126 add 0.473 rmv 0.160 mot 1.000 stuck
room 10 target 137 unla: step  11   37/ 413 points IOU 0.090 add 0.482 rmv 0.010 mot 1.000 stuck
10 121586 points: 407.55s
room 10 NMI: 0.00 AMI: 0.00 ARS: 0.00 PRC: 0.58 RCL: 0.03 IOU: 0.04
(121585,)
121586
Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_10.csv: (121585 points)
Room:  11
room 11 target 143 othe: step 416 120357/3200 points IOU 0.027 add 0.000 rmv 0.031 mot 0.978 noneighbor
room 11 target 197 unla: step  15   93/  93 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 11 target 128 unla: step  24   92/ 378 points IOU 0.243 add 0.000 rmv 1.000 mot 1.000 noe

room 16 target  41 vege: step 436 117202/3129 points IOU 0.027 add 0.000 rmv 0.033 mot 0.971 noneighbor
room 16 target 176 unla: step  11   69/  69 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 16 target 101 unla: step  25  109/ 109 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 16 target 158 unla: step  15  107/ 107 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 16 target 104 unla: step  15   73/  73 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 16 target  88 unla: step  36  382/ 379 points IOU 0.992 add 1.000 rmv 0.994 mot 1.000 noexpand
room 16 target 133 unla: step  16   56/  56 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 16 target 153 unla: step  27  141/ 176 points IOU 0.801 add 0.000 rmv 1.000 mot 1.000 noexpand
room 16 target  61 unla: step   8   54/  54 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 16 target  98 unla: step  17  142/ 145 points IOU 0.979 add 1.000 rmv 1.000 mot

room 22 target  45 unla: step  25  223/ 167 points IOU 0.749 add 0.000 rmv 0.795 mot 1.000 noneighbor
room 22 target  73 unla: step  50  365/ 263 points IOU 0.721 add 1.000 rmv 0.730 mot 1.000 noexpand
room 22 target  44 unla: step  25  560/ 560 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 22 target 125 unla: step  44  179/ 189 points IOU 0.947 add 0.000 rmv 1.000 mot 1.000 noexpand
room 22 target  89 unla: step  17  123/ 123 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 22 target  46 unla: step  16   64/  64 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 22 target 158 unla: step  10   13/  73 points IOU 0.178 add 0.787 rmv 1.000 mot 1.000 stuck
22 108143 points: 438.03s
room 22 NMI: 0.00 AMI: 0.00 ARS: 0.00 PRC: 0.75 RCL: 0.07 IOU: 0.08
(108143,)
108143
Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_22.csv: (108143 points)
Room:  23
room 23 target  97 unla: step  14   24/  61 points IOU 0.393 add 0.404 rmv 0.080 mot 

...

room 209 target   5 movi: step 348 106972/1702 points IOU 0.016 add 0.000 rmv 0.016 mot 0.973 noneighbor
room 209 target  97 vege: step  15  152/ 152 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
209 107133 points: 224.88s
room 209 NMI: 0.00 AMI: 0.00 ARS: 0.00 PRC: 0.50 RCL: 0.01 IOU: 0.02
(107124,)
107133
Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_209.csv: (107124 points)
Room:  210
room 210 target   2 movi: step 349 107636/ 764 points IOU 0.007 add 0.000 rmv 0.008 mot 0.974 noneighbor
room 210 target 114 unla: step  23  123/ 123 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 210 target  97 vege: step  15  161/ 161 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
210 107928 points: 235.37s
room 210 NMI: 0.00 AMI: 0.00 ARS: 0.00 PRC: 0.67 RCL: 0.02 IOU: 0.03
(107920,)
107928
Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_210.csv: (107920 points)
Room:  211
room 211 target   2 movi: step 358 109119/ 786 points

room 221 target 129 unla: step  11   77/  77 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 221 target 126 unla: step  11  212/ 263 points IOU 0.806 add 0.000 rmv 1.000 mot 1.000 noexpand
room 221 target 118 unla: step  14   67/  67 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 221 target 126 unla: step   9   51/ 263 points IOU 0.194 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 221 target  51 vege: step  16  131/ 131 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 221 target 115 unla: step   8  141/ 141 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 221 target 122 unla: step   8  118/ 118 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 221 target  26 vege: step  17  228/ 228 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 221 target 109 unla: step  17  293/ 293 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 221 target 125 unla: step   9   77/  77 points IOU 1.000 add 1.000 rmv

room 230 NMI: 0.00 AMI: 0.00 ARS: 0.00 PRC: 0.75 RCL: 0.02 IOU: 0.03
(131338,)
131348
Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_230.csv: (131338 points)
Room:  231
room 231 target  84 road: step 421 131155/20029 points IOU 0.153 add 0.000 rmv 0.152 mot 0.978 noneighbor
room 231 target 106 vege: step  16  124/ 124 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 231 target 150 unla: step  12   69/  69 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 231 target  34 unla: step  17  285/ 285 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
231 131645 points: 268.53s
room 231 NMI: 0.00 AMI: 0.00 ARS: 0.00 PRC: 0.75 RCL: 0.02 IOU: 0.03
(131633,)
131645
Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_231.csv: (131633 points)
Room:  232
room 232 target  91 road: step 421 132146/20076 points IOU 0.152 add 0.000 rmv 0.135 mot 0.983 noneighbor
room 232 target  36 unla: step  11  247/ 247 points IOU 1.000 add 1.000 rmv 1.

Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_241.csv: (132592 points)
Room:  242
room 242 target 117 road: step 425 131888/  66 points IOU 0.001 add 0.000 rmv 0.000 mot 0.980 noneighbor
room 242 target  64 unla: step  11   65/  65 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 242 target 141 unla: step   9  128/ 128 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 242 target 133 unla: step  24  245/ 245 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 242 target 151 unla: step   8   51/  51 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
room 242 target 148 unla: step   8   73/  73 points IOU 1.000 add 1.000 rmv 1.000 mot 1.000 noneighbor
242 132466 points: 309.72s
room 242 NMI: 0.00 AMI: 0.00 ARS: 0.00 PRC: 0.83 RCL: 0.03 IOU: 0.04
(132450,)
132466
Saved to Z:/PHD/pcseg/results/lrg/MOT_600K_4S_TR_1_LITE_04_0p3/M_242.csv: (132450 points)
Room:  243
room 243 target  58 unla: step 433 131199/ 274 points IOU 0.002 a

OSError: [Errno 22] Invalid argument

In [ ]:
# INFER SEGMENTATION ON A SINGLE KITTI FRAME

import motrackers
import tensorflow as tf
import h5py
import itertools
import time
import scipy
#from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score, adjusted_mutual_info_score

classes_kitti = [''] * 260
classes_kitti[0] = "unlabeled"
classes_kitti[1] = "outlier"
classes_kitti[10] = "car"
classes_kitti[11] = "bicycle"
classes_kitti[13] = "bus"
classes_kitti[15] = "motorcycle"
classes_kitti[16] = "on-rails"
classes_kitti[18] = "truck"
classes_kitti[20] = "other-vehicle"
classes_kitti[30] = "person"
classes_kitti[31] = "bicyclist"
classes_kitti[32] = "motorcyclist"
classes_kitti[40] = "road"
classes_kitti[44] = "parking"
classes_kitti[48] = "sidewalk"
classes_kitti[49] = "other-ground"
classes_kitti[50] = "building"
classes_kitti[51] = "fence"
classes_kitti[52] = "other-structure"
classes_kitti[60] = "lane-marking"
classes_kitti[70] = "vegetation"
classes_kitti[71] = "trunk"
classes_kitti[72] = "terrain"
classes_kitti[80] = "pole"
classes_kitti[81] = "traffic-sign"
classes_kitti[99] = "other-object"
classes_kitti[252] = "moving-car"
classes_kitti[253] = "moving-bicyclist"
classes_kitti[254] = "moving-person"
classes_kitti[255] = "moving-motorcyclist"
classes_kitti[256] = "moving-on-rails"
classes_kitti[257] = "moving-bus"
classes_kitti[258] = "moving-truck"
classes_kitti[259] = "moving-other-vehicle"

agg_nmi = []
agg_ami = []
agg_ars = []
agg_prc = []
agg_rcl = []
agg_iou = []
comp_time_analysis = {
	'feature': [],
	'net': [],
	'neighbor': [],
	'inlier': [],
	'current_net' : [],
	'current_neighbor' : [],
	'current_inlier' : [],
	'iter_net' : [],
	'iter_neighbor' : [],
	'iter_inlier' : [],
}

def saveCSV(filename, points):
	f = open(filename,'w')
	for p in points:
		f.write("%f,%f,%f,%d,%d,%d\n"%(p[0],p[1],p[2],p[3],p[4],p[5]))
	f.close()
	print('Saved to %s: (%d points)'%(filename, len(points)))
    
def saveCSVM(filename, points):
	f = open(filename,'w')
	for p in points:
		f.write("%f,%f,%f,%d,%d,%d,%d\n"%(p[0],p[1],p[2],p[3],p[4],p[5],p[6]))
	f.close()
	print('Saved to %s: (%d points)'%(filename, len(points)))


def loadFromH5ModV2(filename):
	f = h5py.File(filename,'r')
	all_points = f['points'][:]
	count_room = f['count_room'][:]
	tmp_points = []
	idp = 0
	for i in range(len(count_room)):
		tmp_points.append(all_points[idp:idp+count_room[i], :])
		idp += count_room[i]
	f.close()
	room = []
	labels = []
	class_labels = []
	frame_labels = []
	motion_labels = []
	for i in range(len(tmp_points)):
		room.append(tmp_points[i][:,:-5])
		labels.append(tmp_points[i][:,-5].astype(int))
		class_labels.append(tmp_points[i][:,-4].astype(int))
		frame_labels.append(tmp_points[i][:,-3].astype(int))
		motion_labels.append(tmp_points[i][:,-1].astype(int))
	return room, labels, class_labels, frame_labels, motion_labels

# all_points,all_obj_id,all_cls_id,all_frm_id,all_motion = loadFromH5ModV2('Z:\PHD\pcseg\KITTI_MOD\KITTI_I20_03.h5')
all_points,all_obj_id,all_cls_id,all_frm_id,all_motion = loadFromH5ModV2('Z:\PHD\pcseg\KITTI_MOD\KITTI_I20_03.h5')

NUM_INLIER_POINT = 512
NUM_NEIGHBOR_POINT = 512
TRAIN_AREA = [1]
VAL_AREA = [3]
FEATURE_SIZE = 11
LITE = None
resolution = 0.6
add_threshold = 0.5
rmv_threshold = 0.5
cluster_threshold = 10
save_results = True
save_id = 0

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
config.log_device_placement = False
sess = tf.compat.v1.Session(config=config)

net = LrgNet(1, 1, NUM_INLIER_POINT, NUM_NEIGHBOR_POINT, FEATURE_SIZE, LITE)

MODEL_PATH = 'Z:/PHD/pcseg/models3/lrgnet_model%s_xyz_mod.ckpt'%VAL_AREA[0]

saver = tf.compat.v1.train.Saver()
saver.restore(sess, MODEL_PATH)
print('Restored from %s'%MODEL_PATH)

room_id = 17

for repeat_id in range(10):
	print('Room: ',repeat_id)
	unequalized_points = all_points[room_id]
	obj_id = all_obj_id[room_id]
	cls_id = all_cls_id[room_id]
	frm_id = all_frm_id[room_id]
	motion = all_motion[room_id]

	#equalize resolution
	t1 = time.time()
	equalized_idx = []
	unequalized_idx = []
	equalized_map = {}
	normal_grid = {}
	for i in range(len(unequalized_points)):
		k = tuple(numpy.round(unequalized_points[i,:3]/resolution).astype(int))
		if not k in equalized_map:
			equalized_map[k] = len(equalized_idx)
			equalized_idx.append(i)
		unequalized_idx.append(equalized_map[k])
		if not k in normal_grid:
			normal_grid[k] = []
		normal_grid[k].append(i)
	points = unequalized_points[equalized_idx]
	obj_id = obj_id[equalized_idx]
	cls_id = cls_id[equalized_idx]
	frm_id = frm_id[equalized_idx]
	motion = motion[equalized_idx]
	xyz = points[:,:3]
	room_coordinates = (xyz - xyz.min(axis=0)) / (xyz.max(axis=0) - xyz.min(axis=0))

	#compute normals
	normals = []
	curvatures = []
	for i in range(len(points)):
		k = tuple(numpy.round(points[i,:3]/resolution).astype(int))
		neighbors = []
		for offset in itertools.product([-1,0,1],[-1,0,1],[-1,0,1]):
			kk = (k[0]+offset[0], k[1]+offset[1], k[2]+offset[2])
			if kk in normal_grid:
				neighbors.extend(normal_grid[kk])
		accA = numpy.zeros((3,3))
		accB = numpy.zeros(3)
		for n in neighbors:
			p = unequalized_points[n,:3]
			accA += numpy.outer(p,p)
			accB += p
		cov = accA / len(neighbors) - numpy.outer(accB, accB) / len(neighbors)**2
		U,S,V = numpy.linalg.svd(cov)
		normals.append(numpy.fabs(V[2]))
		curvature = S[2] / (S[0] + S[1] + S[2])
		curvatures.append(numpy.fabs(curvature))
	curvatures = numpy.array(curvatures)
	curvatures = curvatures/curvatures.max()
	normals = numpy.array(normals)
	comp_time_analysis['feature'].append(time.time() - t1)
    
	points = numpy.hstack((xyz, room_coordinates, normals, curvatures.reshape(-1,1), frm_id.reshape(-1,1))).astype(numpy.float32)

	point_voxels = numpy.round(points[:,:3]/resolution).astype(int)
	cluster_label = numpy.zeros(len(points), dtype=int)
	motion_label = numpy.zeros(len(points), dtype=int)
	cluster_id = 1
	visited = numpy.zeros(len(point_voxels), dtype=bool)
	inlier_points = numpy.zeros((1, NUM_INLIER_POINT, FEATURE_SIZE), dtype=numpy.float32)
	neighbor_points = numpy.zeros((1, NUM_NEIGHBOR_POINT, FEATURE_SIZE), dtype=numpy.float32)
	input_add = numpy.zeros((1, NUM_NEIGHBOR_POINT), dtype=numpy.int32)
	input_remove = numpy.zeros((1, NUM_INLIER_POINT), dtype=numpy.int32)
	input_motion = numpy.zeros((1, NUM_INLIER_POINT+NUM_NEIGHBOR_POINT), dtype=numpy.int32)
	order = numpy.argsort(curvatures)
	#iterate over each object in the room
	#for seed_id in range(len(point_voxels)):
	for seed_id in numpy.arange(len(points))[order]:
		if visited[seed_id]:
			continue
		seed_voxel = point_voxels[seed_id]
		target_id = obj_id[seed_id]
		target_class = classes_kitti[cls_id[numpy.nonzero(obj_id==target_id)[0][0]]]
		gt_mask = obj_id==target_id
		obj_voxels = point_voxels[gt_mask]
		obj_voxel_set = set([tuple(p) for p in obj_voxels])
		original_minDims = obj_voxels.min(axis=0)
		original_maxDims = obj_voxels.max(axis=0)
		currentMask = numpy.zeros(len(points), dtype=bool)
		currentMask[seed_id] = True
		minDims = seed_voxel.copy()
		maxDims = seed_voxel.copy()
		seqMinDims = minDims
		seqMaxDims = maxDims
		steps = 0
		stuck = 0
		maskLogProb = 0

		#perform region growing
		while True:

			def stop_growing(reason):
				global cluster_id, start_time
				visited[currentMask] = True
				if numpy.sum(currentMask) > cluster_threshold:
					cluster_label[currentMask] = cluster_id
					cluster_id += 1
					iou = 1.0 * numpy.sum(numpy.logical_and(gt_mask,currentMask)) / numpy.sum(numpy.logical_or(gt_mask,currentMask))
					print('room %d target %3d %.4s: step %3d %4d/%4d points IOU %.3f add %.3f rmv %.3f %s'%(room_id, target_id, target_class, steps, numpy.sum(currentMask), numpy.sum(gt_mask), iou, add_acc, rmv_acc, reason))

			#determine the current points and the neighboring points
			t = time.time()
			currentPoints = points[currentMask, :].copy()
			newMinDims = minDims.copy()	
			newMaxDims = maxDims.copy()	
			newMinDims -= 1
			newMaxDims += 1
			mask = numpy.logical_and(numpy.all(point_voxels>=newMinDims,axis=1), numpy.all(point_voxels<=newMaxDims, axis=1))
			mask = numpy.logical_and(mask, numpy.logical_not(currentMask))
			mask = numpy.logical_and(mask, numpy.logical_not(visited))
			expandPoints = points[mask, :].copy()
			expandClass = obj_id[mask] == target_id
			expandClass_motion = motion[mask]
			rejectClass = obj_id[currentMask] != target_id
			rejectClass_motion = motion[currentMask] 

			if len(expandPoints)==0: #no neighbors (early termination)
				stop_growing('noneighbor')
				break

			if len(currentPoints) >= NUM_INLIER_POINT:
				subset = numpy.random.choice(len(currentPoints), NUM_INLIER_POINT, replace=False)
			else:
				subset = list(range(len(currentPoints))) + list(numpy.random.choice(len(currentPoints), NUM_INLIER_POINT-len(currentPoints), replace=True))
			center = numpy.median(currentPoints, axis=0)
			expandPoints = numpy.array(expandPoints)
			expandPoints[:,:2] -= center[:2]
			expandPoints[:,6:] -= center[6:]
			inlier_points[0,:,:] = currentPoints[subset, :]
			inlier_points[0,:,:2] -= center[:2]
			inlier_points[0,:,6:] -= center[6:]
			input_remove[0,:] = numpy.array(rejectClass)[subset]
            
			input_motion[0,:NUM_INLIER_POINT]=numpy.array(rejectClass_motion)[subset]            
            
			if len(expandPoints) >= NUM_NEIGHBOR_POINT:
				subset = numpy.random.choice(len(expandPoints), NUM_NEIGHBOR_POINT, replace=False)
			else:
				subset = list(range(len(expandPoints))) + list(numpy.random.choice(len(expandPoints), NUM_NEIGHBOR_POINT-len(expandPoints), replace=True))
			neighbor_points[0,:,:] = numpy.array(expandPoints)[subset, :]
			input_add[0,:] = numpy.array(expandClass)[subset]
            
			input_motion[0,NUM_INLIER_POINT:]=numpy.array(expandClass_motion)[subset]            
            
			comp_time_analysis['current_neighbor'].append(time.time() - t)
			t = time.time()
			ls, add,add_acc, rmv,rmv_acc, mot,mot_acc = sess.run([net.loss, net.add_output, net.add_acc, net.remove_output, net.remove_acc, net.motion_output, net.motion_acc],
				{net.inlier_pl:inlier_points, net.neighbor_pl:neighbor_points, net.add_mask_pl:input_add, net.remove_mask_pl:input_remove, net.motion_mask_pl:input_motion})
			comp_time_analysis['current_net'].append(time.time() - t)
			t = time.time()

			add_conf = scipy.special.softmax(add[0], axis=-1)[:,1]
			rmv_conf = scipy.special.softmax(rmv[0], axis=-1)[:,1]
			mot_conf = scipy.special.softmax(mot[0], axis=-1)[:,1]
#			add_mask = add_conf > add_threshold
#			rmv_mask = rmv_conf > rmv_threshold
			add_mask = numpy.random.random(len(add_conf)) < add_conf
			rmv_mask = numpy.random.random(len(rmv_conf)) < rmv_conf
			mot_mask = numpy.random.random(len(mot_conf)) < mot_conf
#			add_mask = input_add[0].astype(bool)
#			rmv_mask = input_remove[0].astype(bool)
			addPoints = neighbor_points[0,:,:][add_mask]
			addPoints[:,:2] += center[:2]
			addVoxels = numpy.round(addPoints[:,:3]/resolution).astype(int)
			addSet = set([tuple(p) for p in addVoxels])
			rmvPoints = inlier_points[0,:,:][rmv_mask]
			rmvPoints[:,:2] += center[:2]
			rmvVoxels = numpy.round(rmvPoints[:,:3]/resolution).astype(int)
			rmvSet = set([tuple(p) for p in rmvVoxels])

			motPoints = numpy.vstack((inlier_points[0,:,:],neighbor_points[0,:,:]))[mot_mask]
			motPoints[:,:2] += center[:2]
			motVoxels = numpy.round(motPoints[:,:3]/resolution).astype(int)
			motSet = set([tuple(p) for p in motVoxels])

			updated = False
			iou = 1.0 * numpy.sum(numpy.logical_and(gt_mask,currentMask)) / numpy.sum(numpy.logical_or(gt_mask,currentMask))
#			print('%d/%d points %d outliers %d/%d add %d/%d rmv %.2f iou'%(numpy.sum(numpy.logical_and(currentMask, gt_mask)), numpy.sum(gt_mask),
#				numpy.sum(numpy.logical_and(gt_mask==0, currentMask)), len(addSet), len(expandPoints), len(rmvSet), len(currentPoints), iou))
			for i in range(len(point_voxels)):
				if not currentMask[i] and tuple(point_voxels[i]) in addSet:
					currentMask[i] = True
					updated = True
				if tuple(point_voxels[i]) in rmvSet:
					currentMask[i] = False
				if tuple(point_voxels[i]) in motSet:                    
					motion_label[i] = 1
			steps += 1
			comp_time_analysis['current_inlier'].append(time.time() - t)

			if updated: #continue growing
				minDims = point_voxels[currentMask, :].min(axis=0)
				maxDims = point_voxels[currentMask, :].max(axis=0)
				if not numpy.any(minDims<seqMinDims) and not numpy.any(maxDims>seqMaxDims):
					if stuck >= 1:
						stop_growing('stuck')
						break
					else:
						stuck += 1
				else:
					stuck = 0
				seqMinDims = numpy.minimum(seqMinDims, minDims)
				seqMaxDims = numpy.maximum(seqMaxDims, maxDims)
			else: #no matching neighbors (early termination)
				stop_growing('noexpand')
				break

	#fill in points with no labels
	nonzero_idx = numpy.nonzero(cluster_label)[0]
	nonzero_points = points[nonzero_idx, :]
	filled_cluster_label = cluster_label.copy()
	for i in numpy.nonzero(cluster_label==0)[0]:
		d = numpy.sum((nonzero_points - points[i])**2, axis=1)
		closest_idx = numpy.argmin(d)
		filled_cluster_label[i] = cluster_label[nonzero_idx[closest_idx]]
	cluster_label = filled_cluster_label
	print('%d %d points: %.2fs' % (room_id, len(unequalized_points), time.time() - t1))

	#calculate statistics 
	gt_match = 0
	match_id = 0
	dt_match = numpy.zeros(cluster_label.max(), dtype=bool)
	cluster_label2 = numpy.zeros(len(cluster_label), dtype=int)
	room_iou = []
	unique_id, count = numpy.unique(obj_id, return_counts=True)
	for k in range(len(unique_id)):
		i = unique_id[numpy.argsort(count)][::-1][k]
		best_iou = 0
		for j in range(1, cluster_label.max()+1):
			if not dt_match[j-1]:
				iou = 1.0 * numpy.sum(numpy.logical_and(obj_id==i, cluster_label==j)) / numpy.sum(numpy.logical_or(obj_id==i, cluster_label==j))
				best_iou = max(best_iou, iou)
				if iou > 0.5:
					dt_match[j-1] = True
					gt_match += 1
					cluster_label2[cluster_label==j] = k+1
					break
		room_iou.append(best_iou)
	for j in range(1,cluster_label.max()+1):
		if not dt_match[j-1]:
			cluster_label2[cluster_label==j] = j + obj_id.max()
	prc = numpy.mean(dt_match)
	rcl = 1.0 * gt_match / len(set(obj_id))
	room_iou = numpy.mean(room_iou)

	nmi = 0#normalized_mutual_info_score(obj_id,cluster_label)
	ami = 0#adjusted_mutual_info_score(obj_id,cluster_label)
	ars = 0#adjusted_rand_score(obj_id,cluster_label)
	agg_nmi.append(nmi)
	agg_ami.append(ami)
	agg_ars.append(ars)
	agg_prc.append(prc)
	agg_rcl.append(rcl)
	agg_iou.append(room_iou)
	print("room %d NMI: %.2f AMI: %.2f ARS: %.2f PRC: %.2f RCL: %.2f IOU: %.2f"%(room_id, nmi,ami,ars, prc, rcl, room_iou))

	comp_time_analysis['neighbor'].append(sum(comp_time_analysis['current_neighbor']))
	comp_time_analysis['iter_neighbor'].extend(comp_time_analysis['current_neighbor'])
	comp_time_analysis['current_neighbor'] = []
	comp_time_analysis['net'].append(sum(comp_time_analysis['current_net']))
	comp_time_analysis['iter_net'].extend(comp_time_analysis['current_net'])
	comp_time_analysis['current_net'] = []
	comp_time_analysis['inlier'].append(sum(comp_time_analysis['current_inlier']))
	comp_time_analysis['iter_inlier'].extend(comp_time_analysis['current_inlier'])
	comp_time_analysis['current_inlier'] = []

	print(cluster_label2.shape)
	print(len(unequalized_idx))
	#save point cloud results to file
	if save_results:
		#color_sample_state = numpy.random.RandomState(0)
		#obj_color = color_sample_state.randint(0,255,(numpy.max(cluster_label2)+1,3))
		#obj_color[0] = [100,100,100]
		#unequalized_points[:,3:6] = obj_color[cluster_label2,:][unequalized_idx]
		colors=numpy.random.randint(0,255,(numpy.max(cluster_label)+1,3))
		colors[0] = [100,100,100]
		for i in range(len(points)):
			points[i,3:6] = colors[cluster_label[i],:]
			points[i,6] = motion_label[i]
		#saveCSV('Z:/PHD/pcseg/results/lrg/MOT/%d.csv'%save_id, points)
		saveCSVM('Z:/PHD/pcseg/results/lrg/MOT_300K_4S_TR_1_ROOM17/M_%d.csv'%save_id, points)
		save_id += 1

In [ ]:
# CALCULATE GROUND TRUTH TRACK STATISTICS ON A KITTI SEQUENCE

import numpy as np
import h5py
import sys
import os
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
# import matplotlib.animation as animation
# from PIL import Image
import matplotlib.image

INTERVAL = 20
SKIP = 0
VOXEL_RESOLUTION = 0.3
DOWNSAMPLE_RESOLUTION = 0.3 #0.1
MIN_CLUSTER = 50

sequence='07'

class Track:
    def __init__(self, obj_id):
        self.obj_id = obj_id
        self.centroids = []
        self.frames = []
        self.pointcount = []
        self.length = 0
        self.moving = False
        self.motion = []
        self.averagemotion = 0
        self.bboxs = []
        
    def Extend(self,centroid,frame,pointcount,bbox):
        self.centroids.append(centroid)
        self.frames.append(frame)
        self.pointcount.append(pointcount)
        self.bboxs.append(bbox)
        self.length += 1

def downsample(cloud, resolution):
    voxel_coordinates = [tuple(p) for p in np.round((cloud[:,:3] / resolution)).astype(int)]
    voxel_set = set()
    downsampled_cloud = []
    for i in range(len(cloud)):
        if not voxel_coordinates[i] in voxel_set:
            voxel_set.add(voxel_coordinates[i])
            downsampled_cloud.append(cloud[i])
    return np.array(downsampled_cloud)

def sample_sphere(radius,trans,npts):
    sphere = []
    azis=(np.random.random(npts)-0.5)*2*np.pi
    eles=(np.random.random(npts)-0.5)*np.pi
    for i in range(npts):
        pt=[radius*np.sin(eles[i])*np.cos(azis[i]),radius*np.sin(eles[i])*np.sin(azis[i]),radius*np.cos(eles[i])]
        for j in range(3):
            pt[j]+=trans[j]
        sphere.append(pt)
    return sphere

def sample_cube(side,trans,npts):
    cube = []
    us=np.random.random(npts)-0.5
    vs=np.random.random(npts)-0.5
    faces = np.random.randint(0,6,npts)
    
    rot1=np.random.random()*360
    rot2=np.random.random()*360
    rot3=np.random.random()*360
    r = R.from_euler('zyx', [rot1, rot2, rot3], degrees=True)
    
    for i in range(npts):
        if faces[i]==0:
            pt=[ 0.5*side,us[i]*side,vs[i]*side]
        if faces[i]==1:
            pt=[-0.5*side,us[i]*side,vs[i]*side]
        if faces[i]==2:
            pt=[us[i]*side, 0.5*side,vs[i]*side]
        if faces[i]==3:
            pt=[us[i]*side,-0.5*side,vs[i]*side]
        if faces[i]==4:
            pt=[us[i]*side,vs[i]*side, 0.5*side]
        if faces[i]==5:
            pt=[us[i]*side,vs[i]*side,-0.5*side]
        ptr=r.apply(pt)
        for j in range(3):
            pt[j]+=trans[j]
        cube.append(pt)
    return cube
    

# get camera calibration
calib_file = open('Z:\data_odometry_velodyne\dataset\sequences\\' + sequence + '\calib.txt', 'r')
calib = {}
for line in calib_file:
    key, content = line.strip().split(":")
    values = [float(v) for v in content.strip().split()]
    pose = np.zeros((4, 4))
    pose[0, 0:4] = values[0:4]
    pose[1, 0:4] = values[4:8]
    pose[2, 0:4] = values[8:12]
    pose[3, 3] = 1.0
    calib[key] = pose
calib_file.close()

# get poses
poses = []
Tr = calib["Tr"]
Tr_inv = np.linalg.inv(Tr)
pose_file = open('Z:\data_odometry_velodyne\dataset\sequences\\' + sequence + '\poses.txt', 'r')
for line in pose_file:
    values = [float(v) for v in line.strip().split()]
    pose = np.zeros((4, 4))
    pose[0, 0:4] = values[0:4]
    pose[1, 0:4] = values[4:8]
    pose[2, 0:4] = values[8:12]
    pose[3, 3] = 1.0
    poses.append(np.matmul(Tr_inv, np.matmul(pose, Tr)))
pose_file.close()

scan_paths = 'Z:\data_odometry_velodyne\dataset\sequences\\' + sequence + '\\velodyne'
scan_names = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(scan_paths)) for f in fn]
# scan_names=os.listdir(scan_paths)
scan_names.sort()
print('Scans: ',len(scan_names))
    
label_paths = 'Z:\data_odometry_velodyne\dataset\sequences\\' + sequence + '\labels'
label_names = [os.path.join(dp, f) for dp, dn, fn in os.walk(os.path.expanduser(label_paths)) for f in fn]
label_names.sort()
print('Labels: ',len(label_names))

stacked_points = []
counts = []

frames = len(scan_names)
offset = INTERVAL-1
tracks = []

all_points = []

obj_ids_uq = []
obj_ids_tr = []

colors=np.random.rand(10000,3)*255

# fig1,ax1 = plt.subplots(figsize=(10, 10))
rot1=45
rot2=45
rot3=0
r = R.from_euler('zyx', [rot1, rot2, rot3], degrees=True)
scale = 8
imgsz=1000


for k in range(len(scan_names)):
    
    if k%50 == 0:
        print(k,' of ',len(scan_names))
    scan = np.fromfile(scan_names[k], dtype=np.float32)
    scan = scan.reshape((-1, 4))
    
    # get XYZ in world coordinate frame
    xyz_local = scan[:, 0:3]
    R_world_local = poses[k][:3, :3]
    t_world_local = poses[k][:3, 3]
    xyz_world = xyz_local.dot(R_world_local.T) + t_world_local
#     xyz_voxels = [tuple(v) for v in np.round(xyz_world / VOXEL_RESOLUTION).astype(int)]
    
    # get point labels
    label = np.fromfile(label_names[k], dtype=np.uint32)
    obj_id = [l >> 16 for l in label]
    cls_id = [l & 0xFFFF for l in label]
    
    # stack in Nx5 array
    points = np.zeros((len(xyz_world), 6))
    points[:, :3] = xyz_world
    points[:, 3] = obj_id
    points[:, 4] = cls_id
    
    # BEGIN Image output
    
    rgb=np.zeros((imgsz,imgsz,3),dtype=np.uint8)
    npt = len(xyz_world)
    print(npt)
    
    for w in range(npt):
        cx=xyz_world[w,0]
        cy=xyz_world[w,1]
        cz=xyz_world[w,2]
        xyz_w=np.asarray([cx,cy,cz])
        xyz_l=xyz_w - t_world_local
        #xyz_l=xyz_l.dot(R_local_world.T) 
        
        vr=r.apply([xyz_l[0],xyz_l[1],xyz_l[2]])
        x=int(vr[0]*scale)+int(imgsz/2)
        y=int(vr[1]*scale)+int(imgsz/2)
        if (x>=1) and (x<imgsz-1) and (y>=1) and (y<imgsz-1):
            rr=int(colors[int(points[w,3]),0])
            gg=int(colors[int(points[w,3]),1])
            bb=int(colors[int(points[w,3]),2])
            rgb[x,y,0]=rr
            rgb[x,y,1]=gg
            rgb[x,y,2]=bb
            rgb[x+1,y,0]=rr
            rgb[x+1,y,1]=gg
            rgb[x+1,y,2]=bb
            rgb[x-1,y,0]=rr
            rgb[x-1,y,1]=gg
            rgb[x-1,y,2]=bb
            rgb[x,y+1,0]=rr
            rgb[x,y+1,1]=gg
            rgb[x,y+1,2]=bb
            rgb[x,y-1,0]=rr
            rgb[x,y-1,1]=gg
            rgb[x,y-1,2]=bb
    matplotlib.image.imsave('Z:/PHD/pcseg/TRACKS/'+sequence+'/KITTI_'+sequence+'_'+str(k)+'.png', rgb)
    
    # END Image output
    
    
    
    points_sorted=points[points[:, 3].argsort()]
    obj_id_sorted=points_sorted[:, 3]
    unique_obj_ids=np.unique(obj_id_sorted)
    print(len(unique_obj_ids))
    point_count=0
    
    for obj_id_unique in unique_obj_ids:
        track_index=-1
        for j in range(len(tracks)):
            if(tracks[j].obj_id==obj_id_unique):
                track_index = j
                break
        if track_index == -1:
            track_index = len(tracks)
            tracks.append(Track(obj_id_unique))
            if not(obj_id_unique in obj_ids_uq):
                obj_ids_uq.append(obj_id_unique)
                obj_ids_tr.append(track_index)

        obj_pts=points_sorted[(obj_id_sorted==obj_id_unique).astype(bool),:]
        point_count+=obj_pts.shape[0]
        obj_cent=np.mean(obj_pts, axis=0)
        xmin=np.min(obj_pts[:,0])
        xmax=np.max(obj_pts[:,0])
        ymin=np.min(obj_pts[:,1])
        ymax=np.max(obj_pts[:,1])
        zmin=np.min(obj_pts[:,2])
        zmax=np.max(obj_pts[:,2])
        tracks[track_index].Extend(obj_cent[0:3],k,obj_pts.shape[0],[xmin,ymin,zmin,xmax,ymax,zmax])
        
    fn_m = 'Z:/PHD/pcseg/TRACKS/'+sequence+'_M/M_'+sequence+'_'+str(k)+'.csv'
    file_m=open(fn_m,'w')
        
    for w in range(npt):
        cx=points_sorted[w,0]
        cy=points_sorted[w,1]
        cz=points_sorted[w,2]
        xyz_w=np.asarray([cx,cy,cz])
        xyz_l=xyz_w - t_world_local
        
        obj_id_p = int(points_sorted[w, 3])
        
        strfil='%d,%.4f,%.4f,%.4f'%(obj_id_p,cx,cy,cz)

        file_m.write(strfil)
        file_m.write('\n')
        
    file_m.close()
        
N_TRACKS=len(tracks)

for i in range(N_TRACKS):
    if tracks[i].length >1:
        for j in range(1,tracks[i].length):
            delta = tracks[i].centroids[j]-tracks[i].centroids[j-1]
            tracks[i].motion.append(delta)
            tracks[i].averagemotion+=np.linalg.norm(delta)
        tracks[i].averagemotion/=float(tracks[i].length)
        if tracks[i].averagemotion > 2.5:
            tracks[i].moving = True
        
print(N_TRACKS, ' Unique Tracks found')
for i in range(N_TRACKS):
    if tracks[i].length > 1:
        print('Track: ',i,' with length: ',tracks[i].length,' average speed: ',tracks[i].averagemotion,
              ' average points: ',sum(tracks[i].pointcount)/float(tracks[i].length))
    
        
for u in range(len(obj_ids_uq)):
    print('Object ID: ',obj_ids_uq[u],' is associated with Track: ',obj_ids_tr[u])

file=open('Z:/PHD/pcseg/TRACKS/KITTI_'+sequence+'.csv','w')
for tr in tracks:
    for j in range(tr.length):
        strfil ='%d,%d,%.4f,%.4f,%.4f,%d,%d\n'%(tr.obj_id,j,tr.centroids[j][0],tr.centroids[j][1],tr.centroids[j][2],tr.frames[j],tr.pointcount[j])
        file.write(strfil)
file.close()

file2=open('Z:/PHD/pcseg/TRACKS/KITTI_'+sequence+'_HOTA.csv','w')
for tr in tracks:
    for j in range(tr.length):
        strfil ='%d,%d,%.4f,%.4f,%.4f,%d,%d,%.4f,%.4f,%.4f,%.4f,%.4f,%.4f\n'%(
        tr.obj_id,j,tr.centroids[j][0],tr.centroids[j][1],tr.centroids[j][2],tr.frames[j],tr.pointcount[j],
        tr.bboxs[j][0],tr.bboxs[j][1],tr.bboxs[j][2],tr.bboxs[j][3],tr.bboxs[j][4],tr.bboxs[j][5])
        file2.write(strfil)
file2.close()
    
    